### MODALITY - CARGA DAS BASES DE DADOS POR MODALIDADE

Choose one of the abbreviations in modality variable

Abbreviation | Category
------- | -------
TT | Total
DMS | Demais Créditos
CRT | Cartão de Crédito
VEIC | Financiamento de veículo
FIN | Demais financiamentos
CP | Crédito Pessoal
CHE | Cheque Especial
CNG | Empréstimo Consignado
OEM | Outros Empréstimos
MCC | Microcrédito
VND | Vendor
IMB | Crédito Imobiliário
OCR | Outros Créditos
ADD | Adiantamento a Depositantes
OLM | Outros Limites
RAD | Recebíveis Adquiridos
CGA | Capital de Giro Agricultura

In [1]:
modality = 'TT'

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
8,application_1644856271165_0009,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
lista_cpfs = ['00572740700'] #'00201741873','00323499805','00572740700',

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
bucket = "modelagem"
data_key = "scr/etl-variaveis/raw-data/SCR_MODALIDADE_CLIENTE_2021*.csv"
#data_key = "scr/etl-variaveis/raw-data/extract_SCR_MOD_2.csv"
data_location = f"s3://{bucket}/{data_key}"

scr_data_key = "scr/etl-variaveis/raw-data/SCR_CLIENTE_2021*.csv"
#scr_data_key = "scr/etl-variaveis/raw-data/SCR_CLIENTE.csv"
scr_data_location = f"s3://{bucket}/{scr_data_key}"

#s3://modelagem/scr/omni-scr/
risco_tratado_key = "scr/omni-scr/"
risco_tratado_data_location = f"s3://{bucket}/{risco_tratado_key}"

modalities = {
    'CRT': ['0204','0210','0218','0406','1304','1904'],
    'DMS': ['0404', '0402', '0499', '1399', '0101', '1350', '0802', '0299'],
    'VEIC':['0401'],
    'FIN': ['0402','0499'],
    'CP':  ['0203'],
    'CHE': ['0213','1902'],
    'CNG': ['0202'],
    'OEM': ['0299'],
    #'LCP': ['1906'],
    #'LCC': ['1903'],
    #'LG':  ['1901'],
    'MCC': ['0212', '0403'],
    'VND': ['0404'],
    'IMB': ['0901','0902'],
    'OCR': ['1399'],
    'ADD': ['0101'],
    'OLM': ['1901', '1903', '1906', '1999'],
    'RAD': ['1350'],
    'CGA': ['0802']
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
import botocore.session
import boto3

import pyspark
from pyspark.sql.functions import *
from pyspark.sql.types import StructType,StructField, StringType, IntegerType 
from pyspark.sql.types import ArrayType, DoubleType, BooleanType
from pyspark.sql.functions import col,array_contains,months_between,sum
from pyspark.sql import functions as F
from operator import add, and_, sub
from functools import reduce

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
def sumIgnoringNaN(listOfColumnsToSum):
    return F.when(
        reduce(and_,
               [(F.col(column).isNull()) for column in listOfColumnsToSum]),
        F.lit(None)).otherwise(
            reduce(add, [
                F.coalesce(F.col(column), F.lit(0.0))
                for column in listOfColumnsToSum
            ]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
def avgIgnoringNaN(listOfColumnsToAvg):
    return F.when(
                reduce(and_, [(F.col(column).isNull()) for column in listOfColumnsToAvg]), 
                F.lit(None)
            ).otherwise(
                #the first reduce does the sum ignoring nan by replacing it by 0, the second reduce counts the number of non nan values in order to do the avg ignoring nan
                reduce(add, [F.coalesce(F.col(column), F.lit(0.0)) for column in listOfColumnsToAvg])/reduce(add, [F.col(column).isNotNull().cast(IntegerType()) for column in listOfColumnsToAvg])
            )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
def subIgnoringNaN(listOfColumnsToSub):
    return F.when(
        reduce(and_,
               [(F.col(column).isNull()) for column in listOfColumnsToSub]),
        F.lit(None)).otherwise(
            reduce(sub, [
                F.coalesce(F.col(column), F.lit(0.0))
                for column in listOfColumnsToSub
            ]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
schema = StructType()\
      .add("DTBASE",IntegerType(),True)\
      .add("CD",StringType(),True)\
      .add("TP",StringType(),True)\
      .add("MOD",StringType(),True)\
      .add("VINCME",StringType(),True)\
      .add("LIMITE_CRED_VENCIMENTO_ATE_360",StringType(),True)\
      .add("LIMITE_CRED_VENCIM_ACIMA_360",StringType(),True)\
      .add("CRED_A_LIBERAR_ATE_360",StringType(),True)\
      .add("CRED_A_LIBERAR_ACIMA_360",StringType(),True)\
      .add("CRED_A_VENCER_ATE_30",StringType(),True)\
      .add("CRED_A_VENCER_DE_31_A_60",StringType(),True)\
      .add("CRED_A_VENCER_DE_61_A_90",StringType(),True)\
      .add("CRED_A_VENCER_DE_91_A_180",StringType(),True)\
      .add("CRED_A_VENCER_DE_181_A_360",StringType(),True)\
      .add("CRED_A_VENCER_DE_361_A_720",StringType(),True)\
      .add("CRED_A_VENCER_DE_721_A_1080",StringType(),True)\
      .add("CRED_A_VENCER_DE_1081_A_1440",StringType(),True)\
      .add("CRED_A_VENCER_DE_1441_A_1800",StringType(),True)\
      .add("CRED_A_VENCER_DE_1801_A_5400",StringType(),True)\
      .add("CRED_A_VENCER_ACIMA_5400",StringType(),True)\
      .add("CRED_A_VENCER_PRAZO_INDETERM",StringType(),True)\
      .add("CRED_VENCIDOS_DE_1_A_14",StringType(),True)\
      .add("CRED_VENCIDOS_DE_15_A_30",StringType(),True)\
      .add("CRED_VENCIDOS_DE_31_A_60",StringType(),True)\
      .add("CRED_VENCIDOS_DE_61_A_90",StringType(),True)\
      .add("CRED_VENCIDOS_DE_91_A_120",StringType(),True)\
      .add("CRED_VENCIDOS_DE_121_A_150",StringType(),True)\
      .add("CRED_VENCIDOS_DE_151_A_180",StringType(),True)\
      .add("CRED_VENCIDOS_DE_181_A_240",StringType(),True)\
      .add("CRED_VENCIDOS_DE_241_A_300",StringType(),True)\
      .add("CRED_VENCIDOS_DE_301_A_360",StringType(),True)\
      .add("CRED_VENCIDOS_DE_361_A_540",StringType(),True)\
      .add("CRED_VENCIDOS_ACIMA_540",StringType(),True)\
      .add("CRED_BAIX_PREJ_ATE_12_M",StringType(),True)\
      .add("CRED_BAIX_PREJ_MAIS_12_A_48_M",StringType(),True)\
      .add("CRED_BAIX_PREJ_MAIS_48_M",StringType(),True)\
      .add("ORIGEM",StringType(),True)\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
#Leitura da base SCR_CLIENTE_MODALIDADE
df_SCR_CLIENTE_MODALIDADE = spark.read.format("csv") \
      .options(header='True', delimiter=';') \
      .schema(schema) \
      .load(data_location)

#converting string to double by replacing ',' by '.' and then casting
for column in df_SCR_CLIENTE_MODALIDADE.columns:
    if column.startswith('LIMITE') or column.startswith('CRED'):
        df_SCR_CLIENTE_MODALIDADE = df_SCR_CLIENTE_MODALIDADE.withColumn(column, F.regexp_replace(column, ',','.')).withColumn(column, F.col(column).cast(DoubleType()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
schema_cliente = StructType() \
      .add("DTBASE",StringType(),True) \
      .add("CD",StringType(),True) \
      .add("TP",StringType(),True) \
      .add("COOBASS",StringType(),True) \
      .add("COOBREC",StringType(),True) \
      .add("INIRELACTCLI",StringType(),True) \
      .add("QTDIF",StringType(),True) \
      .add("QTDOPMANIF",StringType(),True) \
      .add("QTDOPJUD",StringType(),True) \
      .add("QTDOP",StringType(),True) \
      .add("RESPTOTMANIF",StringType(),True) \
      .add("RESPTOTJUD",StringType(),True) \
      .add("RISCOINDVENDOR",StringType(),True) \
      .add("ORIGEM",StringType(),True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
#Leitura da base SCR_CLIENTE
df_SCR_CLIENTE = spark.read.format("csv") \
      .options(header='True', delimiter=';') \
      .schema(schema_cliente) \
      .load(scr_data_location)

#converting string to double by replacing ',' by '.' and then casting
for column in df_SCR_CLIENTE.columns:
    if column.startswith('LIMITE') or column.startswith('CRED'):
        df_SCR_CLIENTE = df_SCR_CLIENTE.withColumn(column, F.regexp_replace(column, ',','.')).withColumn(column, F.col(column).cast(DoubleType()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
df_risco_tratado = spark.read.parquet(risco_tratado_data_location)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
df_risco_tratado = df_risco_tratado.withColumn('CRED_A_VENCER_ATE_30', sumIgnoringNaN(['CRED_A_VENCER_ATE_30','CRED_VENCIDOS_DE_1_A_14']))
df_risco_tratado = df_risco_tratado.withColumn('CRED_VENCIDOS_DE_1_A_14', F.lit(0))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
df_risco_tratado = df_risco_tratado.withColumn('DATA_BASE', substring(regexp_replace(col('DATA_BASE'), "-", ""),1,6).cast(StringType()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
lista_divida = ["CRED_A_VENCER_ATE_30",
"CRED_A_VENCER_DE_31_A_60",
"CRED_A_VENCER_DE_61_A_90",
"CRED_A_VENCER_DE_91_A_180",
"CRED_A_VENCER_DE_181_A_360",
"CRED_A_VENCER_DE_361_A_720",
"CRED_A_VENCER_DE_721_A_1080",
"CRED_A_VENCER_DE_1081_A_1440",
"CRED_A_VENCER_DE_1441_A_1800",
"CRED_A_VENCER_DE_1801_A_5400",
"CRED_A_VENCER_ACIMA_5400",
"CRED_A_VENCER_PRAZO_INDETERM",
"CRED_VENCIDOS_DE_1_A_14",
"CRED_VENCIDOS_DE_15_A_30",
"CRED_VENCIDOS_DE_31_A_60",
"CRED_VENCIDOS_DE_61_A_90",
"CRED_VENCIDOS_DE_91_A_120",
"CRED_VENCIDOS_DE_121_A_150",
"CRED_VENCIDOS_DE_151_A_180",
"CRED_VENCIDOS_DE_181_A_240",
"CRED_VENCIDOS_DE_241_A_300",
"CRED_VENCIDOS_DE_301_A_360",
"CRED_VENCIDOS_DE_361_A_540",
"CRED_VENCIDOS_ACIMA_540",
"CRED_BAIX_PREJ_ATE_12_M",
"CRED_BAIX_PREJ_MAIS_12_A_48_M",
"CRED_BAIX_PREJ_MAIS_48_M"]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
lista_vencidos = ['CRED_VENCIDOS_DE_1_A_14',
'CRED_VENCIDOS_DE_15_A_30',
'CRED_VENCIDOS_DE_31_A_60',
'CRED_VENCIDOS_DE_61_A_90',
'CRED_VENCIDOS_DE_91_A_120',
'CRED_VENCIDOS_DE_121_A_150',
'CRED_VENCIDOS_DE_151_A_180',
'CRED_VENCIDOS_DE_181_A_240',
'CRED_VENCIDOS_DE_241_A_300',
'CRED_VENCIDOS_DE_301_A_360',
'CRED_VENCIDOS_DE_361_A_540',
'CRED_VENCIDOS_ACIMA_540']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
lista_vencer = ['CRED_A_VENCER_ATE_30',
'CRED_A_VENCER_DE_31_A_60',
'CRED_A_VENCER_DE_61_A_90',
'CRED_A_VENCER_DE_91_A_180',
'CRED_A_VENCER_DE_181_A_360',
'CRED_A_VENCER_DE_361_A_720',
'CRED_A_VENCER_DE_721_A_1080',
'CRED_A_VENCER_DE_1081_A_1440',
'CRED_A_VENCER_DE_1441_A_1800',
'CRED_A_VENCER_DE_1801_A_5400',
'CRED_A_VENCER_ACIMA_5400',
'CRED_A_VENCER_PRAZO_INDETERM']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
lista_prejuizo = ['CRED_BAIX_PREJ_ATE_12_M',
'CRED_BAIX_PREJ_MAIS_12_A_48_M',
'CRED_BAIX_PREJ_MAIS_48_M']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
#renaming columns to remove the sum(column) aggregation in the name
for column in df_risco_tratado.columns:
    df_risco_tratado = df_risco_tratado.withColumnRenamed(column, column.replace('sum(','').replace(')',''))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
df_risco_tratado = df_risco_tratado.withColumn(f'{modality}_SALDO_TOTAL_TT',round(sumIgnoringNaN(lista_divida).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
df_risco_tratado = df_risco_tratado.withColumn(f'{modality}_SALDO_VCD_TOTAL_TT',round(sumIgnoringNaN(lista_vencidos).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
df_risco_tratado = df_risco_tratado.withColumn(f'{modality}_SALDO_AVCR_TOTAL_TT',round(sumIgnoringNaN(lista_vencer).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
df_risco_tratado = df_risco_tratado.withColumn(f'{modality}_SALDO_PREJUIZO_TOTAL_TT',round(sumIgnoringNaN(lista_prejuizo).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
df_risco_tratado = df_risco_tratado.withColumnRenamed('DATA_BASE','DTBASE')
df_risco_tratado = df_risco_tratado.withColumnRenamed('CPF_CNPJ','CD')
df_risco_tratado = df_risco_tratado.withColumnRenamed('MODALIDADE_OPERACAO','MOD')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
df_risco_tratado = df_risco_tratado.na.fill(value=0)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
for column in df_risco_tratado.columns:
    if column.startswith('LIMITE') or column.startswith('CRED'):
        df_risco_tratado = df_risco_tratado.withColumn(column, F.round(column, 4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
df_risco_tratado_market = df_risco_tratado

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
df_risco_tratado_market = df_risco_tratado_market.filter(df_risco_tratado_market.CD.isin(lista_cpfs))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
df_SCR_CLIENTE_MODALIDADE = df_SCR_CLIENTE_MODALIDADE.filter(df_SCR_CLIENTE_MODALIDADE.CD.isin(lista_cpfs))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
for column in df_risco_tratado_market.columns[3:]:
    df_risco_tratado_market = df_risco_tratado_market.withColumnRenamed(column, column + '_OMNI')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
df_visao_omni = df_risco_tratado_market.select("DTBASE","CD", "MOD", f"{modality}_SALDO_TOTAL_TT_OMNI",f"{modality}_SALDO_VCD_TOTAL_TT_OMNI",f"{modality}_SALDO_AVCR_TOTAL_TT_OMNI",f"{modality}_SALDO_PREJUIZO_TOTAL_TT_OMNI")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
df_lista_duplicados = df_SCR_CLIENTE_MODALIDADE.groupBy("DTBASE","CD", "MOD").agg(count('CD').alias('count')).filter(col("count") > 1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
cpfs_drop = df_lista_duplicados.withColumn('ORIGEM',when(df_lista_duplicados.DTBASE > 0,'TRIGG'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
df2 = df_SCR_CLIENTE_MODALIDADE.na.fill(value=0)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
df2 = df2.join(cpfs_drop,["DTBASE","CD","MOD","ORIGEM"],"leftanti")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
df2 = df2.withColumn(f'{modality}_SALDO_TOTAL_TT',round(sumIgnoringNaN(lista_divida).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
df2 = df2.withColumn(f'{modality}_SALDO_VCD_TOTAL_TT',round(sumIgnoringNaN(lista_vencidos).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
df2 = df2.withColumn(f'{modality}_SALDO_AVCR_TOTAL_TT',round(sumIgnoringNaN(lista_vencer).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
df2 = df2.withColumn(f'{modality}_SALDO_PREJUIZO_TOTAL_TT',round(sumIgnoringNaN(lista_prejuizo).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
df2 = df2.groupBy("DTBASE","CD", "MOD").sum().drop('sum(DTBASE)', 'sum(TP)')

#renaming columns to remove the sum(column) aggregation in the name
for column in df2.columns:
    df2 = df2.withColumnRenamed(column, column.replace('sum(','').replace(')',''))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
df2 = df2.join(df_risco_tratado_market,["DTBASE","CD", "MOD"],"left")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
#dropping unused columns
df2 = df2.drop('TP','VINCME','ORIGEM')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
df2.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- DTBASE: integer (nullable = true)
 |-- CD: string (nullable = true)
 |-- MOD: string (nullable = true)
 |-- LIMITE_CRED_VENCIMENTO_ATE_360: double (nullable = true)
 |-- LIMITE_CRED_VENCIM_ACIMA_360: double (nullable = true)
 |-- CRED_A_LIBERAR_ATE_360: double (nullable = true)
 |-- CRED_A_LIBERAR_ACIMA_360: double (nullable = true)
 |-- CRED_A_VENCER_ATE_30: double (nullable = true)
 |-- CRED_A_VENCER_DE_31_A_60: double (nullable = true)
 |-- CRED_A_VENCER_DE_61_A_90: double (nullable = true)
 |-- CRED_A_VENCER_DE_91_A_180: double (nullable = true)
 |-- CRED_A_VENCER_DE_181_A_360: double (nullable = true)
 |-- CRED_A_VENCER_DE_361_A_720: double (nullable = true)
 |-- CRED_A_VENCER_DE_721_A_1080: double (nullable = true)
 |-- CRED_A_VENCER_DE_1081_A_1440: double (nullable = true)
 |-- CRED_A_VENCER_DE_1441_A_1800: double (nullable = true)
 |-- CRED_A_VENCER_DE_1801_A_5400: double (nullable = true)
 |-- CRED_A_VENCER_ACIMA_5400: double (nullable = true)
 |-- CRED_A_VENCER_PRAZ

In [44]:
#the when logic is necessary to keep nan if the columns are all null, otherwise it will have a value
for column in df2.columns[3:38]:
    df2 = df2.withColumn(column+'_MERCADO',
        F.when(
            (F.col(column).isNull() & F.col(column+'_OMNI').isNull()),
            F.lit(None)
        ).otherwise(
            F.round(F.coalesce(F.col(column),F.lit(0.0))-F.coalesce(F.col(column+'_OMNI'),F.lit(0.0)),4))
        ).drop(column+'_OMNI').withColumnRenamed(column, column+'_TOTAL')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
df2 = df2.join(df_visao_omni,["DTBASE","CD","MOD"],"left")#.persist()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
df2 = df2.withColumn('MENOR_VALOR_VISAO_MERCADO', F.least(
                                                                F.col('CRED_A_VENCER_ATE_30_MERCADO'),
                                                                F.col('CRED_A_VENCER_DE_31_A_60_MERCADO'),
                                                                F.col('CRED_A_VENCER_DE_61_A_90_MERCADO'),
                                                                F.col('CRED_A_VENCER_DE_91_A_180_MERCADO'),
                                                                F.col('CRED_A_VENCER_DE_181_A_360_MERCADO'),
                                                                F.col('CRED_A_VENCER_DE_361_A_720_MERCADO'),
                                                                F.col('CRED_A_VENCER_DE_721_A_1080_MERCADO'),
                                                                F.col('CRED_A_VENCER_DE_1081_A_1440_MERCADO'),
                                                                F.col('CRED_A_VENCER_DE_1441_A_1800_MERCADO'),
                                                                F.col('CRED_A_VENCER_DE_1801_A_5400_MERCADO'),
                                                                F.col('CRED_A_VENCER_ACIMA_5400_MERCADO'),
                                                                F.col('CRED_A_VENCER_PRAZO_INDETERM_MERCADO'),
                                                                F.col('CRED_VENCIDOS_DE_1_A_14_MERCADO'),
                                                                F.col('CRED_VENCIDOS_DE_15_A_30_MERCADO'),
                                                                F.col('CRED_VENCIDOS_DE_31_A_60_MERCADO'),
                                                                F.col('CRED_VENCIDOS_DE_61_A_90_MERCADO'),
                                                                F.col('CRED_VENCIDOS_DE_91_A_120_MERCADO'),
                                                                F.col('CRED_VENCIDOS_DE_121_A_150_MERCADO'),
                                                                F.col('CRED_VENCIDOS_DE_151_A_180_MERCADO'),
                                                                F.col('CRED_VENCIDOS_DE_181_A_240_MERCADO'),
                                                                F.col('CRED_VENCIDOS_DE_241_A_300_MERCADO'),
                                                                F.col('CRED_VENCIDOS_DE_301_A_360_MERCADO'),
                                                                F.col('CRED_VENCIDOS_DE_361_A_540_MERCADO'),
                                                                F.col('CRED_VENCIDOS_ACIMA_540_MERCADO'),
                                                                F.col('CRED_BAIX_PREJ_ATE_12_M_MERCADO'),
                                                                F.col('CRED_BAIX_PREJ_MAIS_12_A_48_M_MERCADO'),
                                                                F.col('CRED_BAIX_PREJ_MAIS_48_M_MERCADO'),
                                                                F.col('LIMITE_CRED_VENCIMENTO_ATE_360_MERCADO'),
                                                                F.col('LIMITE_CRED_VENCIM_ACIMA_360_MERCADO')
                                                            ))#ajustar - visao total manter - eliminar (zerar) somente visao mercado ou nao demonstrar - deixar somente as de mercado negativo e manter as de total

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [47]:
if modality!='TT':
    df2 = df2.filter(df2.MOD.isin(modalities[modality]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
df2 = df2.groupBy("DTBASE","CD").sum().drop('sum(DTBASE)', 'sum(TP)')

#renaming columns to remove the sum(column) aggregation in the name
for column in df2.columns:
    df2 = df2.withColumnRenamed(column, column.replace('sum(','').replace(')',''))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [49]:
df2 = df2.na.fill(value=0)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
df2 = df2.filter(df2.CD.isin(lista_cpfs))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
df2.show()

In [51]:
df2 = df2.withColumn('CRED_A_VENCER_ATE_30_MERCADO', when(df2.MENOR_VALOR_VISAO_MERCADO < 0,0).otherwise(df2.CRED_A_VENCER_ATE_30_MERCADO))
df2 = df2.withColumn('CRED_A_VENCER_DE_31_A_60_MERCADO', when(df2.MENOR_VALOR_VISAO_MERCADO < 0,0).otherwise(df2.CRED_A_VENCER_DE_31_A_60_MERCADO))
df2 = df2.withColumn('CRED_A_VENCER_DE_61_A_90_MERCADO', when(df2.MENOR_VALOR_VISAO_MERCADO < 0,0).otherwise(df2.CRED_A_VENCER_DE_61_A_90_MERCADO))
df2 = df2.withColumn('CRED_A_VENCER_DE_91_A_180_MERCADO', when(df2.MENOR_VALOR_VISAO_MERCADO < 0,0).otherwise(df2.CRED_A_VENCER_DE_91_A_180_MERCADO))
df2 = df2.withColumn('CRED_A_VENCER_DE_181_A_360_MERCADO', when(df2.MENOR_VALOR_VISAO_MERCADO < 0,0).otherwise(df2.CRED_A_VENCER_DE_181_A_360_MERCADO))
df2 = df2.withColumn('CRED_A_VENCER_DE_361_A_720_MERCADO', when(df2.MENOR_VALOR_VISAO_MERCADO < 0,0).otherwise(df2.CRED_A_VENCER_DE_361_A_720_MERCADO))
df2 = df2.withColumn('CRED_A_VENCER_DE_721_A_1080_MERCADO', when(df2.MENOR_VALOR_VISAO_MERCADO < 0,0).otherwise(df2.CRED_A_VENCER_DE_721_A_1080_MERCADO))
df2 = df2.withColumn('CRED_A_VENCER_DE_1081_A_1440_MERCADO', when(df2.MENOR_VALOR_VISAO_MERCADO < 0,0).otherwise(df2.CRED_A_VENCER_DE_1081_A_1440_MERCADO))
df2 = df2.withColumn('CRED_A_VENCER_DE_1441_A_1800_MERCADO', when(df2.MENOR_VALOR_VISAO_MERCADO < 0,0).otherwise(df2.CRED_A_VENCER_DE_1441_A_1800_MERCADO))
df2 = df2.withColumn('CRED_A_VENCER_DE_1801_A_5400_MERCADO', when(df2.MENOR_VALOR_VISAO_MERCADO < 0,0).otherwise(df2.CRED_A_VENCER_DE_1801_A_5400_MERCADO))
df2 = df2.withColumn('CRED_A_VENCER_ACIMA_5400_MERCADO', when(df2.MENOR_VALOR_VISAO_MERCADO < 0,0).otherwise(df2.CRED_A_VENCER_ACIMA_5400_MERCADO))
df2 = df2.withColumn('CRED_A_VENCER_PRAZO_INDETERM_MERCADO', when(df2.MENOR_VALOR_VISAO_MERCADO < 0,0).otherwise(df2.CRED_A_VENCER_PRAZO_INDETERM_MERCADO))
df2 = df2.withColumn('CRED_VENCIDOS_DE_1_A_14_MERCADO', when(df2.MENOR_VALOR_VISAO_MERCADO < 0,0).otherwise(df2.CRED_VENCIDOS_DE_1_A_14_MERCADO))
df2 = df2.withColumn('CRED_VENCIDOS_DE_15_A_30_MERCADO', when(df2.MENOR_VALOR_VISAO_MERCADO < 0,0).otherwise(df2.CRED_VENCIDOS_DE_15_A_30_MERCADO))
df2 = df2.withColumn('CRED_VENCIDOS_DE_31_A_60_MERCADO', when(df2.MENOR_VALOR_VISAO_MERCADO < 0,0).otherwise(df2.CRED_VENCIDOS_DE_31_A_60_MERCADO))
df2 = df2.withColumn('CRED_VENCIDOS_DE_61_A_90_MERCADO', when(df2.MENOR_VALOR_VISAO_MERCADO < 0,0).otherwise(df2.CRED_VENCIDOS_DE_61_A_90_MERCADO))
df2 = df2.withColumn('CRED_VENCIDOS_DE_91_A_120_MERCADO', when(df2.MENOR_VALOR_VISAO_MERCADO < 0,0).otherwise(df2.CRED_VENCIDOS_DE_91_A_120_MERCADO))
df2 = df2.withColumn('CRED_VENCIDOS_DE_121_A_150_MERCADO', when(df2.MENOR_VALOR_VISAO_MERCADO < 0,0).otherwise(df2.CRED_VENCIDOS_DE_121_A_150_MERCADO))
df2 = df2.withColumn('CRED_VENCIDOS_DE_151_A_180_MERCADO', when(df2.MENOR_VALOR_VISAO_MERCADO < 0,0).otherwise(df2.CRED_VENCIDOS_DE_151_A_180_MERCADO))
df2 = df2.withColumn('CRED_VENCIDOS_DE_181_A_240_MERCADO', when(df2.MENOR_VALOR_VISAO_MERCADO < 0,0).otherwise(df2.CRED_VENCIDOS_DE_181_A_240_MERCADO))
df2 = df2.withColumn('CRED_VENCIDOS_DE_241_A_300_MERCADO', when(df2.MENOR_VALOR_VISAO_MERCADO < 0,0).otherwise(df2.CRED_VENCIDOS_DE_241_A_300_MERCADO))
df2 = df2.withColumn('CRED_VENCIDOS_DE_301_A_360_MERCADO', when(df2.MENOR_VALOR_VISAO_MERCADO < 0,0).otherwise(df2.CRED_VENCIDOS_DE_301_A_360_MERCADO))
df2 = df2.withColumn('CRED_VENCIDOS_DE_361_A_540_MERCADO', when(df2.MENOR_VALOR_VISAO_MERCADO < 0,0).otherwise(df2.CRED_VENCIDOS_DE_361_A_540_MERCADO))
df2 = df2.withColumn('CRED_VENCIDOS_ACIMA_540_MERCADO', when(df2.MENOR_VALOR_VISAO_MERCADO < 0,0).otherwise(df2.CRED_VENCIDOS_ACIMA_540_MERCADO))
df2 = df2.withColumn('CRED_BAIX_PREJ_ATE_12_M_MERCADO', when(df2.MENOR_VALOR_VISAO_MERCADO < 0,0).otherwise(df2.CRED_BAIX_PREJ_ATE_12_M_MERCADO))
df2 = df2.withColumn('CRED_BAIX_PREJ_MAIS_12_A_48_M_MERCADO', when(df2.MENOR_VALOR_VISAO_MERCADO < 0,0).otherwise(df2.CRED_BAIX_PREJ_MAIS_12_A_48_M_MERCADO))
df2 = df2.withColumn('CRED_BAIX_PREJ_MAIS_48_M_MERCADO', when(df2.MENOR_VALOR_VISAO_MERCADO < 0,0).otherwise(df2.CRED_BAIX_PREJ_MAIS_48_M_MERCADO))
df2 = df2.withColumn('LIMITE_CRED_VENCIMENTO_ATE_360_MERCADO', when(df2.MENOR_VALOR_VISAO_MERCADO < 0,0).otherwise(df2.LIMITE_CRED_VENCIMENTO_ATE_360_MERCADO))
df2 = df2.withColumn('LIMITE_CRED_VENCIM_ACIMA_360_MERCADO', when(df2.MENOR_VALOR_VISAO_MERCADO < 0,0).otherwise(df2.LIMITE_CRED_VENCIM_ACIMA_360_MERCADO))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [52]:
import datetime
from dateutil.relativedelta import relativedelta

df2 = df2.withColumn("M0",col("DTBASE").cast(StringType()))
df2 = df2.withColumn("M0",to_date(col("M0"),"yyyyMM"))

df2 = df2.withColumn("M5",add_months(date_format(col("M0"),'yyyy-MM-dd'),-5))
df2 = df2.withColumn("M4",add_months(date_format(col("M0"),'yyyy-MM-dd'),-4))
df2 = df2.withColumn("M3",add_months(date_format(col("M0"),'yyyy-MM-dd'),-3))
df2 = df2.withColumn("M2",add_months(date_format(col("M0"),'yyyy-MM-dd'),-2))
df2 = df2.withColumn("M1",add_months(date_format(col("M0"),'yyyy-MM-dd'),-1))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [53]:
for column in df2.columns:
    if column.startswith(f'{modality}_'):
        df2 = df2.withColumn(column, F.round(column, 4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

MAIOR_ATRASO_VCD & MAIOR_ATRASO_VCD_MERC

In [54]:
#Maior atraso de crédito vencido
from pyspark.sql.functions import when
df4 = df2.withColumn(f"{modality}_MAIOR_ATRASO_VCD", when((df2.CRED_VENCIDOS_ACIMA_540_TOTAL > 0),999)
                                          .when((df2.CRED_VENCIDOS_DE_361_A_540_TOTAL > 0),540)
                                          .when((df2.CRED_VENCIDOS_DE_301_A_360_TOTAL > 0),360)
                                          .when((df2.CRED_VENCIDOS_DE_241_A_300_TOTAL > 0),300)
                                          .when((df2.CRED_VENCIDOS_DE_181_A_240_TOTAL > 0),240)
                                          .when((df2.CRED_VENCIDOS_DE_151_A_180_TOTAL > 0),180)
                                          .when((df2.CRED_VENCIDOS_DE_121_A_150_TOTAL > 0),150)
                                          .when((df2.CRED_VENCIDOS_DE_91_A_120_TOTAL > 0),120)
                                          .when((df2.CRED_VENCIDOS_DE_61_A_90_TOTAL > 0),90)
                                          .when((df2.CRED_VENCIDOS_DE_31_A_60_TOTAL > 0),60)
                                          .when((df2.CRED_VENCIDOS_DE_15_A_30_TOTAL > 0),30)
                                          .when((df2.CRED_VENCIDOS_DE_1_A_14_TOTAL > 0),14)
                                          .otherwise(0))

df4 = df4.withColumn(f"{modality}_MAIOR_ATRASO_VCD_MERC", F.when(df4.CRED_VENCIDOS_ACIMA_540_MERCADO > 0,999)
                                                 .when(df4.CRED_VENCIDOS_DE_361_A_540_MERCADO > 0,540)
                                                 .when(df4.CRED_VENCIDOS_DE_301_A_360_MERCADO > 0,360)
                                                 .when(df4.CRED_VENCIDOS_DE_241_A_300_MERCADO > 0,300)
                                                 .when(df4.CRED_VENCIDOS_DE_181_A_240_MERCADO > 0,240)
                                                 .when(df4.CRED_VENCIDOS_DE_151_A_180_MERCADO > 0,180)
                                                 .when(df4.CRED_VENCIDOS_DE_121_A_150_MERCADO > 0,150)
                                                 .when(df4.CRED_VENCIDOS_DE_91_A_120_MERCADO > 0,120)
                                                 .when(df4.CRED_VENCIDOS_DE_61_A_90_MERCADO > 0,90)
                                                 .when(df4.CRED_VENCIDOS_DE_31_A_60_MERCADO > 0,60)
                                                 .when(df4.CRED_VENCIDOS_DE_15_A_30_MERCADO > 0,30)
                                                 .when(df4.CRED_VENCIDOS_DE_1_A_14_MERCADO > 0,14)
                                                 .otherwise(0))#verificar zerado e nao nulo

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
lista_divida = ["CRED_A_VENCER_ATE_30",
"CRED_A_VENCER_DE_31_A_60",
"CRED_A_VENCER_DE_61_A_90",
"CRED_A_VENCER_DE_91_A_180",
"CRED_A_VENCER_DE_181_A_360",
"CRED_A_VENCER_DE_361_A_720",
"CRED_A_VENCER_DE_721_A_1080",
"CRED_A_VENCER_DE_1081_A_1440",
"CRED_A_VENCER_DE_1441_A_1800",
"CRED_A_VENCER_DE_1801_A_5400",
"CRED_A_VENCER_ACIMA_5400",
"CRED_A_VENCER_PRAZO_INDETERM",
"CRED_VENCIDOS_DE_1_A_14",
"CRED_VENCIDOS_DE_15_A_30",
"CRED_VENCIDOS_DE_31_A_60",
"CRED_VENCIDOS_DE_61_A_90",
"CRED_VENCIDOS_DE_91_A_120",
"CRED_VENCIDOS_DE_121_A_150",
"CRED_VENCIDOS_DE_151_A_180",
"CRED_VENCIDOS_DE_181_A_240",
"CRED_VENCIDOS_DE_241_A_300",
"CRED_VENCIDOS_DE_301_A_360",
"CRED_VENCIDOS_DE_361_A_540",
"CRED_VENCIDOS_ACIMA_540",
"CRED_BAIX_PREJ_ATE_12_M",
"CRED_BAIX_PREJ_MAIS_12_A_48_M",
"CRED_BAIX_PREJ_MAIS_48_M"]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [56]:
lista_vencidos_total = ['CRED_VENCIDOS_DE_1_A_14_TOTAL',
'CRED_VENCIDOS_DE_15_A_30_TOTAL',
'CRED_VENCIDOS_DE_31_A_60_TOTAL',
'CRED_VENCIDOS_DE_61_A_90_TOTAL',
'CRED_VENCIDOS_DE_91_A_120_TOTAL',
'CRED_VENCIDOS_DE_121_A_150_TOTAL',
'CRED_VENCIDOS_DE_151_A_180_TOTAL',
'CRED_VENCIDOS_DE_181_A_240_TOTAL',
'CRED_VENCIDOS_DE_241_A_300_TOTAL',
'CRED_VENCIDOS_DE_301_A_360_TOTAL',
'CRED_VENCIDOS_DE_361_A_540_TOTAL',
'CRED_VENCIDOS_ACIMA_540_TOTAL']

lista_vencidos_mercado = ['CRED_VENCIDOS_DE_1_A_14_MERCADO',
'CRED_VENCIDOS_DE_15_A_30_MERCADO',
'CRED_VENCIDOS_DE_31_A_60_MERCADO',
'CRED_VENCIDOS_DE_61_A_90_MERCADO',
'CRED_VENCIDOS_DE_91_A_120_MERCADO',
'CRED_VENCIDOS_DE_121_A_150_MERCADO',
'CRED_VENCIDOS_DE_151_A_180_MERCADO',
'CRED_VENCIDOS_DE_181_A_240_MERCADO',
'CRED_VENCIDOS_DE_241_A_300_MERCADO',
'CRED_VENCIDOS_DE_301_A_360_MERCADO',
'CRED_VENCIDOS_DE_361_A_540_MERCADO',
'CRED_VENCIDOS_ACIMA_540_MERCADO']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [57]:
lista_vencer_total = ['CRED_A_VENCER_ATE_30_TOTAL',
'CRED_A_VENCER_DE_31_A_60_TOTAL',
'CRED_A_VENCER_DE_61_A_90_TOTAL',
'CRED_A_VENCER_DE_91_A_180_TOTAL',
'CRED_A_VENCER_DE_181_A_360_TOTAL',
'CRED_A_VENCER_DE_361_A_720_TOTAL',
'CRED_A_VENCER_DE_721_A_1080_TOTAL',
'CRED_A_VENCER_DE_1081_A_1440_TOTAL',
'CRED_A_VENCER_DE_1441_A_1800_TOTAL',
'CRED_A_VENCER_DE_1801_A_5400_TOTAL',
'CRED_A_VENCER_ACIMA_5400_TOTAL',
'CRED_A_VENCER_PRAZO_INDETERM_TOTAL']

lista_vencer_mercado = ['CRED_A_VENCER_ATE_30_MERCADO',
'CRED_A_VENCER_DE_31_A_60_MERCADO',
'CRED_A_VENCER_DE_61_A_90_MERCADO',
'CRED_A_VENCER_DE_91_A_180_MERCADO',
'CRED_A_VENCER_DE_181_A_360_MERCADO',
'CRED_A_VENCER_DE_361_A_720_MERCADO',
'CRED_A_VENCER_DE_721_A_1080_MERCADO',
'CRED_A_VENCER_DE_1081_A_1440_MERCADO',
'CRED_A_VENCER_DE_1441_A_1800_MERCADO',
'CRED_A_VENCER_DE_1801_A_5400_MERCADO',
'CRED_A_VENCER_ACIMA_5400_MERCADO',
'CRED_A_VENCER_PRAZO_INDETERM_MERCADO']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [58]:
lista_prejuizo_total = ['CRED_BAIX_PREJ_ATE_12_M_TOTAL',
'CRED_BAIX_PREJ_MAIS_12_A_48_M_TOTAL',
'CRED_BAIX_PREJ_MAIS_48_M_TOTAL']

lista_prejuizo_mercado = ['CRED_BAIX_PREJ_ATE_12_M_MERCADO',
'CRED_BAIX_PREJ_MAIS_12_A_48_M_MERCADO',
'CRED_BAIX_PREJ_MAIS_48_M_MERCADO']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

VCD & VCD_MERC & VCD_RZMO

In [59]:
#Obtenção da soma de todas as colunas de crédito vencido e a vencer em uma coluna de resposta para cada
df4 = df4.withColumn(f'{modality}_VCD',round(reduce(add, [F.col(x) for x in lista_vencidos_total]).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [60]:
#Obtenção da soma de todas as colunas de crédito vencido e a vencer em uma coluna de resposta para cada
df4 = df4.withColumn(f'{modality}_VCD_MERC',round(reduce(add, [F.col(x) for x in lista_vencidos_mercado]).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [61]:
#Obtenção da soma de todas as colunas de crédito vencido e a vencer em uma coluna de resposta para cada - PONDERADO
df4 = df4.withColumn(f'{modality}_VCD_RZMO',round(F.col(f'{modality}_VCD_MERC')/F.col(f'{modality}_SALDO_TOTAL_TT_OMNI').cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [62]:
#AQUI - HOMOLOGAÇÃO

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [63]:
df4 = df4.filter(df4.CD.isin(lista_cpfs))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [64]:
df4.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- DTBASE: integer (nullable = true)
 |-- CD: string (nullable = true)
 |-- LIMITE_CRED_VENCIMENTO_ATE_360_TOTAL: double (nullable = false)
 |-- LIMITE_CRED_VENCIM_ACIMA_360_TOTAL: double (nullable = false)
 |-- CRED_A_LIBERAR_ATE_360_TOTAL: double (nullable = false)
 |-- CRED_A_LIBERAR_ACIMA_360_TOTAL: double (nullable = false)
 |-- CRED_A_VENCER_ATE_30_TOTAL: double (nullable = false)
 |-- CRED_A_VENCER_DE_31_A_60_TOTAL: double (nullable = false)
 |-- CRED_A_VENCER_DE_61_A_90_TOTAL: double (nullable = false)
 |-- CRED_A_VENCER_DE_91_A_180_TOTAL: double (nullable = false)
 |-- CRED_A_VENCER_DE_181_A_360_TOTAL: double (nullable = false)
 |-- CRED_A_VENCER_DE_361_A_720_TOTAL: double (nullable = false)
 |-- CRED_A_VENCER_DE_721_A_1080_TOTAL: double (nullable = false)
 |-- CRED_A_VENCER_DE_1081_A_1440_TOTAL: double (nullable = false)
 |-- CRED_A_VENCER_DE_1441_A_1800_TOTAL: double (nullable = false)
 |-- CRED_A_VENCER_DE_1801_A_5400_TOTAL: double (nullable = false)
 |-- CRED_A_VENC

In [65]:
#df4.select("CD","DTBASE","TT_SALDO_TOTAL_TT_TOTAL","TT_SALDO_TOTAL_TT_MERCADO","TT_SALDO_TOTAL_TT_OMNI").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

AVCR & AVCR_MERC & AVCR_RZMO

In [66]:
#Obtenção da soma de todas as colunas de crédito vencido e a vencer em uma coluna de resposta para cada
df4 = df4.withColumn(f'{modality}_AVCR',round(reduce(add, [F.col(x) for x in lista_vencer_total]).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [67]:
#Obtenção da soma de todas as colunas de crédito vencido e a vencer em uma coluna de resposta para cada
df4 = df4.withColumn(f'{modality}_AVCR_MERC',round(reduce(add, [F.col(x) for x in lista_vencer_mercado]).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [68]:
#Obtenção da soma de todas as colunas de crédito vencido e a vencer em uma coluna de resposta para cada
df4 = df4.withColumn(f'{modality}_AVCR_RZMO',round(F.col(f'{modality}_AVCR')/F.col(f'{modality}_SALDO_TOTAL_TT_OMNI').cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

RZ_VCD_POR_VCD_E_AVCR & RZ_VCD_POR_VCD_E_AVCR_MERC

In [69]:
#(Crédito vencido)/(Crédito vencido + Crédito a vencer)
#Crédito vencido + Crédito a vencer
df4 = df4.withColumn(f'{modality}_RZ_VCD_POR_VCD_E_AVCR',round(F.col(f'{modality}_VCD')/(F.col(f'{modality}_VCD')+F.col(f'{modality}_AVCR')).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [70]:
#(Crédito vencido)/(Crédito vencido + Crédito a vencer)
#Crédito vencido + Crédito a vencer
df4 = df4.withColumn(f'{modality}_RZ_VCD_POR_VCD_E_AVCR_MERC',round(F.col(f'{modality}_VCD_MERC')/(F.col(f'{modality}_VCD_MERC')+F.col(f'{modality}_AVCR_MERC')).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

VCD_E_AVCR_TOTAL & VCD_E_AVCR_TOTAL_MERC & VCD_E_AVCR_TOTAL_RZMO

In [71]:
#(Crédito vencido)/(Crédito vencido + Crédito a vencer)
#Crédito vencido + Crédito a vencer
df4 = df4.withColumn(f'{modality}_VCD_E_AVCR_TOTAL',round((F.col(f'{modality}_VCD')+F.col(f'{modality}_AVCR')).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [72]:
#(Crédito vencido)/(Crédito vencido + Crédito a vencer)
#Crédito vencido + Crédito a vencer
df4 = df4.withColumn(f'{modality}_VCD_E_AVCR_MERC',round((F.col(f'{modality}_VCD_MERC')+F.col(f'{modality}_AVCR_MERC')).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [73]:
#(Crédito vencido)/(Crédito vencido + Crédito a vencer) - PONDERADO
df4 = df4.withColumn(f'{modality}_VCD_E_AVCR_RZMO',round((F.col(f'{modality}_VCD_E_AVCR_MERC')/F.col(f'{modality}_SALDO_TOTAL_TT_OMNI')).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [74]:
#building temporary columns 
df4 = df4.withColumn(f'{modality}_PREJ', sumIgnoringNaN(lista_prejuizo_total))
df4 = df4.withColumn(f'{modality}_PREJ_MERC', sumIgnoringNaN(lista_prejuizo_mercado))
df4 = df4.withColumn(f'{modality}_VCD_E_AVCR_E_PREJ', sumIgnoringNaN([f'{modality}_VCD',f'{modality}_AVCR',f'{modality}_PREJ']))
df4 = df4.withColumn(f'{modality}_VCD_E_AVCR_E_PREJ_MERC', sumIgnoringNaN([f'{modality}_VCD_MERC',f'{modality}_AVCR_MERC',f'{modality}_PREJ_MERC']))
df4 = df4.withColumn(f'{modality}_VCD_E_PREJ', sumIgnoringNaN([f'{modality}_VCD',f'{modality}_PREJ']))
df4 = df4.withColumn(f'{modality}_VCD_E_PREJ_MERC', sumIgnoringNaN([f'{modality}_VCD_MERC',f'{modality}_PREJ_MERC']))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

RZ_VCD_PREJ_POR_DIVIDA & RZ_VCD_PREJ_POR_DIVIDA_MERC

In [75]:
df4 = df4.withColumn(f'{modality}_RZ_VCD_PREJ_POR_DIVIDA', round(F.col(f'{modality}_VCD_E_PREJ')/F.col(f'{modality}_VCD_E_AVCR_E_PREJ').cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [76]:
df4 = df4.withColumn(f'{modality}_RZ_VCD_PREJ_POR_DIVIDA_MERC', round(F.col(f'{modality}_VCD_E_PREJ_MERC')/F.col(f'{modality}_VCD_E_AVCR_E_PREJ_MERC').cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

PREJ_ATE_12M & PREJ_ATE_12M_MERC & PREJ_ATE_12M_RZMO

In [77]:
#Baixa de prejuízo até 12M
df4 = df4.withColumn(f'{modality}_PREJ_ATE_12M',round((df4.CRED_BAIX_PREJ_ATE_12_M_TOTAL).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [78]:
#Baixa de prejuízo até 12M
df4 = df4.withColumn(f'{modality}_PREJ_ATE_12M_MERC',round((df4.CRED_BAIX_PREJ_ATE_12_M_MERCADO).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [79]:
#Baixa de prejuízo até 12M - PONDERADO
df4 = df4.withColumn(f'{modality}_PREJ_ATE_12M_RZMO',round(F.col(f'{modality}_PREJ_ATE_12M_MERC')/F.col(f'{modality}_SALDO_TOTAL_TT_OMNI').cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

PREJ_12_48M & PREJ_12_48M_MERC & PREJ_12_48M_RZMO

In [80]:
#Baixa de prejuízo entre 12M e 48M
df4 = df4.withColumn(f'{modality}_PREJ_12_48M',round((df4.CRED_BAIX_PREJ_MAIS_12_A_48_M_TOTAL).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [81]:
#Baixa de prejuízo entre 12M e 48M
df4 = df4.withColumn(f'{modality}_PREJ_12_48M_MERC',round((df4.CRED_BAIX_PREJ_MAIS_12_A_48_M_MERCADO).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [82]:
#Baixa de prejuízo entre 12M e 48M - PONDERADO
df4 = df4.withColumn(f'{modality}_PREJ_12_48M_RZMO',round(F.col(f'{modality}_PREJ_12_48M_MERC')/F.col(f'{modality}_SALDO_TOTAL_TT_OMNI').cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [83]:
df4.select("CD", "DTBASE",f'{modality}_PREJ_12_48M', f'{modality}_SALDO_TOTAL_TT_OMNI',f'{modality}_PREJ_12_48M_RZMO').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+------+--------------+----------------------+-------------------+
|         CD|DTBASE|TT_PREJ_12_48M|TT_SALDO_TOTAL_TT_OMNI|TT_PREJ_12_48M_RZMO|
+-----------+------+--------------+----------------------+-------------------+
|00572740700|202109|        726.64|                   0.0|               null|
|00572740700|202111|           0.0|                   0.0|               null|
|00572740700|202108|        726.64|                   0.0|               null|
|00572740700|202110|           0.0|                   0.0|               null|
|00572740700|202107|        726.64|                   0.0|               null|
|00572740700|202106|       1268.59|                541.95|             1.3408|
+-----------+------+--------------+----------------------+-------------------+

In [84]:
#df4.select("CD","DTBASE",f'{modality}_PREJ_12_48M_MERC',"CRED_BAIX_PREJ_MAIS_12_A_48_M_MERCADO","CRED_BAIX_PREJ_MAIS_12_A_48_M_TOTAL",f'{modality}_PREJ_MERC').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [85]:
#Limite disponível - Modalidade 1904
lista_limite_disponivel = ['LIMITE_CRED_VENCIMENTO_ATE_360_TOTAL',
'LIMITE_CRED_VENCIM_ACIMA_360_TOTAL']

lista_limite_disponivel_mercado = ['LIMITE_CRED_VENCIMENTO_ATE_360_MERCADO',
'LIMITE_CRED_VENCIM_ACIMA_360_MERCADO']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [86]:
#Limite total - Modalidade 1904
lista_limite_total = ['LIMITE_CRED_VENCIMENTO_ATE_360_TOTAL',
'LIMITE_CRED_VENCIM_ACIMA_360_TOTAL',
'CRED_VENCIDOS_DE_1_A_14_TOTAL',
'CRED_VENCIDOS_DE_15_A_30_TOTAL',
'CRED_VENCIDOS_DE_31_A_60_TOTAL',
'CRED_VENCIDOS_DE_61_A_90_TOTAL',
'CRED_VENCIDOS_DE_91_A_120_TOTAL',
'CRED_VENCIDOS_DE_121_A_150_TOTAL',
'CRED_VENCIDOS_DE_151_A_180_TOTAL',
'CRED_VENCIDOS_DE_181_A_240_TOTAL',
'CRED_VENCIDOS_DE_241_A_300_TOTAL',
'CRED_VENCIDOS_DE_301_A_360_TOTAL',
'CRED_VENCIDOS_DE_361_A_540_TOTAL',
'CRED_VENCIDOS_ACIMA_540_TOTAL',
'CRED_A_VENCER_ATE_30_TOTAL',
'CRED_A_VENCER_DE_31_A_60_TOTAL',
'CRED_A_VENCER_DE_61_A_90_TOTAL',
'CRED_A_VENCER_DE_91_A_180_TOTAL',
'CRED_A_VENCER_DE_181_A_360_TOTAL',
'CRED_A_VENCER_DE_361_A_720_TOTAL',
'CRED_A_VENCER_DE_721_A_1080_TOTAL',
'CRED_A_VENCER_DE_1081_A_1440_TOTAL',
'CRED_A_VENCER_DE_1441_A_1800_TOTAL',
'CRED_A_VENCER_DE_1801_A_5400_TOTAL',
'CRED_A_VENCER_ACIMA_5400_TOTAL',
'CRED_A_VENCER_PRAZO_INDETERM_TOTAL']


lista_limite_total_mercado = ['LIMITE_CRED_VENCIMENTO_ATE_360_MERCADO',
'LIMITE_CRED_VENCIM_ACIMA_360_MERCADO',
'CRED_VENCIDOS_DE_1_A_14_MERCADO',
'CRED_VENCIDOS_DE_15_A_30_MERCADO',
'CRED_VENCIDOS_DE_31_A_60_MERCADO',
'CRED_VENCIDOS_DE_61_A_90_MERCADO',
'CRED_VENCIDOS_DE_91_A_120_MERCADO',
'CRED_VENCIDOS_DE_121_A_150_MERCADO',
'CRED_VENCIDOS_DE_151_A_180_MERCADO',
'CRED_VENCIDOS_DE_181_A_240_MERCADO',
'CRED_VENCIDOS_DE_241_A_300_MERCADO',
'CRED_VENCIDOS_DE_301_A_360_MERCADO',
'CRED_VENCIDOS_DE_361_A_540_MERCADO',
'CRED_VENCIDOS_ACIMA_540_MERCADO',
'CRED_A_VENCER_ATE_30_MERCADO',
'CRED_A_VENCER_DE_31_A_60_MERCADO',
'CRED_A_VENCER_DE_61_A_90_MERCADO',
'CRED_A_VENCER_DE_91_A_180_MERCADO',
'CRED_A_VENCER_DE_181_A_360_MERCADO',
'CRED_A_VENCER_DE_361_A_720_MERCADO',
'CRED_A_VENCER_DE_721_A_1080_MERCADO',
'CRED_A_VENCER_DE_1081_A_1440_MERCADO',
'CRED_A_VENCER_DE_1441_A_1800_MERCADO',
'CRED_A_VENCER_DE_1801_A_5400_MERCADO',
'CRED_A_VENCER_ACIMA_5400_MERCADO',
'CRED_A_VENCER_PRAZO_INDETERM_MERCADO']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

LMT_DISPONIVEL & LMT_DISPONIVEL_MERC & LMT_DISPONIVEL_RZMO

In [87]:
#soma das colunas de limite disponivel para inclusão de valor por cpf na respectiva coluna de limite dosponivel
df4 = df4.withColumn(f'{modality}_LMT_DISPONIVEL',round(reduce(add, [F.col(x) for x in lista_limite_disponivel]).cast(DoubleType()),4))
#soma das colunas de limite disponivel para inclusão de valor por cpf na respectiva coluna de limite dosponivel
df4 = df4.withColumn(f'{modality}_LMT_DISPONIVEL_MERC',round(reduce(add, [F.col(x) for x in lista_limite_disponivel_mercado]).cast(DoubleType()),4))
#soma das colunas de limite disponivel para inclusão de valor por cpf na respectiva coluna de limite dosponivel - PONDERADO
df4 = df4.withColumn(f'{modality}_LMT_DISPONIVEL_RZMO',round(F.col(f'{modality}_LMT_DISPONIVEL_MERC')/F.col(f'{modality}_SALDO_TOTAL_TT_OMNI').cast(DoubleType()),4))
    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

LMT_TOTAL & LMT_TOTAL_MERC & LMT_TOTAL_RZMO

In [88]:
#soma das colunas de limite para inclusão de valor de limite total considerando os critérios
df4 = df4.withColumn(f'{modality}_LMT_TOTAL',when((F.col(f'{modality}_VCD') + df4.CRED_BAIX_PREJ_ATE_12_M_TOTAL + df4.CRED_BAIX_PREJ_MAIS_12_A_48_M_TOTAL) == 0,round(reduce(add, [F.col(x) for x in lista_limite_total]).cast(DoubleType()),4)).otherwise(0.0).cast(DoubleType()))
#soma das colunas de limite para inclusão de valor de limite total considerando os critérios
df4 = df4.withColumn(f'{modality}_LMT_TOTAL_MERC',when((F.col(f'{modality}_VCD') + df4.CRED_BAIX_PREJ_ATE_12_M_MERCADO + df4.CRED_BAIX_PREJ_MAIS_12_A_48_M_MERCADO) == 0,round(reduce(add, [F.col(x) for x in lista_limite_total_mercado]).cast(DoubleType()),4)).otherwise(0.0).cast(DoubleType()))
#soma das colunas de limite para inclusão de valor de limite total considerando os critérios - PONDERADO
df4 = df4.withColumn(f'{modality}_LMT_TOTAL_RZMO',round((F.col(f'{modality}_LMT_TOTAL_MERC')/F.col(f'{modality}_SALDO_TOTAL_TT_OMNI')).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [89]:
s3 = boto3.resource('s3')
s3.Object('modelagem', f"s3://modelagem/scr/m0-processed-data/df4/{modality}").delete()

print(f"/scr/m0-processed-data/df4/{modality}/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/scr/m0-processed-data/df4/TT/

In [90]:
df4.write.format("parquet").mode("Overwrite").save(f"s3://modelagem/scr/m0-processed-data/df4/{modality}/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [91]:
df_consolida = spark.read.parquet(f"s3://modelagem/scr/m0-processed-data/df4/{modality}/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [92]:
lista_limite_total_modalidade = ['CRT', 'CHE']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [93]:
lista_limite_disponivel=['CRT', 'CHE', 'OLM']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

QTD_IFS & QTD_OPS

In [94]:
#Select para que seja obtido somente o maior atraso de credito vencido por cpf e sinalizado
df9 = df_SCR_CLIENTE.select("DTBASE","CD","INIRELACTCLI","QTDIF","QTDOP").distinct()#.persist()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [95]:
#(Limite total)/(Total de Instituições Financeiras)
df10 = df_consolida.join(df9,["DTBASE","CD"],"left")#.persist()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [96]:
del(df2)
del(df4)
del(df9)
del(df_consolida)
del(df_SCR_CLIENTE_MODALIDADE)
del(df_SCR_CLIENTE)
del(df_risco_tratado)
del(df_risco_tratado_market)
del(df_visao_omni)
del(df_lista_duplicados)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [97]:
df10 = df10.filter(df10.CD.isin(lista_cpfs))#homologação

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [98]:
df10.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

6

TEMPO_RELACIONAMENTO

In [99]:
df10 = df10.withColumn(f'{modality}_TEMPO_RELACIONAMENTO',F.when(df10.INIRELACTCLI.isNotNull(), months_between(col("M0"),col("INIRELACTCLI")).cast(IntegerType())).otherwise(0))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

RZ_LMT_TOTAL_QTD_INST_FIN & RZ_LMT_TOTAL_QTD_OPERAC

In [100]:
if modality in lista_limite_total_modalidade:
    #(Limite total)/(Total de Instituições Financeiras)
    df10 = df10.withColumn(f'{modality}_RZ_LMT_TOTAL_QTD_INST_FIN',round(F.col(f'{modality}_LMT_TOTAL')/df10.QTDIF.cast(DoubleType()),4))
    #(Limite total)/(Total de Operações)
    df10 = df10.withColumn(f'{modality}_RZ_LMT_TOTAL_QTD_OPERAC',round(F.col(f'{modality}_LMT_TOTAL')/df10.QTDOP.cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

PERC_LIMITE_DISPONIVEL & PERC_LIMITE_DISPONIVEL_MERC

In [101]:
if modality in lista_limite_total_modalidade:
    #percentual Limite disponivel
    df10 = df10.withColumn(f'{modality}_PERC_LIMITE_DISPONIVEL',round(F.col(f'{modality}_LMT_DISPONIVEL')/F.col(f'{modality}_LMT_TOTAL').cast(DoubleType()),4))
    #percentual Limite disponivel 
    df10 = df10.withColumn(f'{modality}_PERC_LIMITE_DISPONIVEL_MERC',round(F.col(f'{modality}_LMT_DISPONIVEL_MERC')/F.col(f'{modality}_LMT_TOTAL_MERC').cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

MEDIA_AVCR_3MESES & MEDIA_AVCR_3MESES_MERC

In [102]:
lista_vencer_3meses = ['CRED_A_VENCER_ATE_30_TOTAL',
'CRED_A_VENCER_DE_31_A_60_TOTAL',
'CRED_A_VENCER_DE_61_A_90_TOTAL']

lista_vencer_3meses_mercado = ['CRED_A_VENCER_ATE_30_MERCADO',
'CRED_A_VENCER_DE_31_A_60_MERCADO',
'CRED_A_VENCER_DE_61_A_90_MERCADO']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [103]:
#Média dos vencimentos dos próximos 3 meses V2
df10 = df10.withColumn(f'{modality}_MEDIA_AVCR_3MESES',round((reduce(add, [F.col(x) for x in lista_vencer_3meses])/3).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [104]:
#Média dos vencimentos dos próximos 3 meses V2
df10 = df10.withColumn(f'{modality}_MEDIA_AVCR_3MESES_MERC',round((reduce(add, [F.col(x) for x in lista_vencer_3meses_mercado])/3).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

MEDIA_AVCR_3MESES_V2 & MEDIA_AVCR_3MESES_MERC_V2 & _MEDIA_AVCR_3MESES_RZMO

In [105]:
#Média dos vencimentos dos próximos 3 meses
df10 = df10.withColumn(f'{modality}_MEDIA_AVCR_3MESES_V2', round(avgIgnoringNaN(lista_vencer_3meses).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [106]:
#Média dos vencimentos dos próximos 3 meses MERCADO
df10 = df10.withColumn(f'{modality}_MEDIA_AVCR_3MESES_MERC_V2', round(avgIgnoringNaN(lista_vencer_3meses_mercado).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [107]:
#Média dos vencimentos dos próximos 3 meses - PONDERADO VER VER VER VER VER VER VER VER VER VER VER 
df10 = df10.withColumn(f'{modality}_MEDIA_AVCR_3MESES_RZMO',round(F.col(f'{modality}_MEDIA_AVCR_3MESES_MERC')/F.col(f'{modality}_SALDO_TOTAL_TT_OMNI').cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [108]:
#(Média dos vencimentos dos próximos 3 meses)/(Soma das parcelas dos contratos ativos do cliente) VER VER VER VER VER VER VER VER VER VER VER 
df10 = df10.withColumn(f'{modality}_MEDIA_AVCR_3MESES_RZMO_POR_M0',round(F.col(f'{modality}_MEDIA_AVCR_3MESES')/df10.CRED_A_VENCER_ATE_30_TOTAL.cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [109]:
#(Média dos vencimentos dos próximos 3 meses)/(Soma das parcelas dos contratos ativos do cliente) VER VER VER VER VER VER VER VER VER VER VER 
df10 = df10.withColumn(f'{modality}_MEDIA_AVCR_3MESES_RZMO_POR_M0_MERC',round(F.col(f'{modality}_MEDIA_AVCR_3MESES_MERC')/df10.CRED_A_VENCER_ATE_30_MERCADO.cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

MAIOR_PRAZO_AVCR & MAIOR_PRAZO_AVCR_MERC

In [110]:
#Maior vencimento de crédito 
df10 = df10.withColumn(f"{modality}_MAIOR_PRAZO_AVCR", when((df10.CRED_A_VENCER_ACIMA_5400_TOTAL > 0),5499)
                                      .when((df10.CRED_A_VENCER_DE_1801_A_5400_TOTAL > 0),5400)
                                      .when((df10.CRED_A_VENCER_DE_1441_A_1800_TOTAL > 0),1800)
                                      .when((df10.CRED_A_VENCER_DE_1081_A_1440_TOTAL > 0),1440)
                                      .when((df10.CRED_A_VENCER_DE_721_A_1080_TOTAL > 0),1080)
                                      .when((df10.CRED_A_VENCER_DE_361_A_720_TOTAL > 0),720)
                                      .when((df10.CRED_A_VENCER_DE_181_A_360_TOTAL > 0),360)
                                      .when((df10.CRED_A_VENCER_DE_91_A_180_TOTAL > 0),180)
                                      .when((df10.CRED_A_VENCER_DE_61_A_90_TOTAL > 0),90)
                                      .when((df10.CRED_A_VENCER_DE_31_A_60_TOTAL > 0),60)
                                      .when((df10.CRED_A_VENCER_ATE_30_TOTAL > 0),30)
                                      .otherwise(0))

df10 = df10.withColumn(f"{modality}_MAIOR_PRAZO_AVCR_MERC", when((df10.CRED_A_VENCER_ACIMA_5400_MERCADO > 0),5499)
                                      .when((df10.CRED_A_VENCER_DE_1801_A_5400_MERCADO > 0),5400)
                                      .when((df10.CRED_A_VENCER_DE_1441_A_1800_MERCADO > 0),1800)
                                      .when((df10.CRED_A_VENCER_DE_1081_A_1440_MERCADO > 0),1440)
                                      .when((df10.CRED_A_VENCER_DE_721_A_1080_MERCADO > 0),1080)
                                      .when((df10.CRED_A_VENCER_DE_361_A_720_MERCADO > 0),720)
                                      .when((df10.CRED_A_VENCER_DE_181_A_360_MERCADO > 0),360)
                                      .when((df10.CRED_A_VENCER_DE_91_A_180_MERCADO > 0),180)
                                      .when((df10.CRED_A_VENCER_DE_61_A_90_MERCADO > 0),90)
                                      .when((df10.CRED_A_VENCER_DE_31_A_60_MERCADO > 0),60)
                                      .when((df10.CRED_A_VENCER_ATE_30_MERCADO > 0),30)
                                      .otherwise(0))#Ver a questão do zero

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

RZ_AVCR_ATE90_POR_AVCR & RZ_AVCR_ATE90_POR_AVCR_MERC Á RZ_AVCR_ACI1800_POR_AVCR & RZ_AVCR_ACI1800_POR_AVCR_MERC

In [111]:
#(Crédito a vencer até 90 dias)/(Total de crédito a vencer)
df10 = df10.withColumn(f'{modality}_RZ_AVCR_ATE90_POR_AVCR',round((reduce(add, [F.col(x) for x in lista_vencer_3meses])/F.col(f'{modality}_AVCR')).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [112]:
#(Crédito a vencer até 90 dias)/(Total de crédito a vencer)

#CORRIGIR NOMENCLATURA A VENCER

df10 = df10.withColumn(f'{modality}_RZ_AVCR_ATE90_POR_AVCR_MERC',round((reduce(add, [F.col(x) for x in lista_vencer_3meses_mercado])/F.col(f'{modality}_AVCR_MERC')).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [113]:
#(Crédito a vencer entre 91 e 180 dias)/(Total de crédito a vencer) 
df10 = df10.withColumn(f'{modality}_RZ_AVCR_91_180_POR_AVCR',round((df10.CRED_A_VENCER_DE_91_A_180_TOTAL/F.col(f'{modality}_AVCR')).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [114]:
#(Crédito a vencer entre 91 e 180 dias)/(Total de crédito a vencer)
df10 = df10.withColumn(f'{modality}_RZ_AVCR_91_180_POR_AVCR_MERC',round((df10.CRED_A_VENCER_DE_91_A_180_MERCADO/F.col(f'{modality}_AVCR_MERC')).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [115]:
#(Crédito a vencer entre 181 e 360 dias)/(Total de crédito a vencer)
df10 = df10.withColumn(f'{modality}_RZ_AVCR_181_360_POR_AVCR',round((df10.CRED_A_VENCER_DE_181_A_360_TOTAL/F.col(f'{modality}_AVCR')).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [116]:
#(Crédito a vencer entre 181 e 360 dias)/(Total de crédito a vencer)
df10 = df10.withColumn(f'{modality}_RZ_AVCR_181_360_POR_AVCR_MERC',round((df10.CRED_A_VENCER_DE_181_A_360_MERCADO/F.col(f'{modality}_AVCR_MERC')).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [117]:
#(Crédito a vencer entre 361 e 720 dias)/(Total de crédito a vencer)
df10 = df10.withColumn(f'{modality}_RZ_AVCR_361_720_POR_AVCR',round((df10.CRED_A_VENCER_DE_361_A_720_TOTAL/F.col(f'{modality}_AVCR')).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [118]:
#(Crédito a vencer entre 361 e 720 dias)/(Total de crédito a vencer)
df10 = df10.withColumn(f'{modality}_RZ_AVCR_361_720_POR_AVCR_MERC',round((df10.CRED_A_VENCER_DE_361_A_720_MERCADO/F.col(f'{modality}_AVCR_MERC')).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [119]:
#(Crédito a vencer entre 721 e 1080 dias)/(Total de crédito a vencer)
df10 = df10.withColumn(f'{modality}_RZ_AVCR_721_1080_POR_AVCR',round((df10.CRED_A_VENCER_DE_721_A_1080_TOTAL/F.col(f'{modality}_AVCR')).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [120]:
#(Crédito a vencer entre 721 e 1080 dias)/(Total de crédito a vencer)
df10 = df10.withColumn(f'{modality}_RZ_AVCR_721_1080_POR_AVCR_MERC',round((df10.CRED_A_VENCER_DE_721_A_1080_MERCADO/F.col(f'{modality}_AVCR_MERC')).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [121]:
lista_1080_1800 = ['CRED_A_VENCER_DE_1081_A_1440_TOTAL',
'CRED_A_VENCER_DE_1441_A_1800_TOTAL']

lista_1080_1800_mercado = ['CRED_A_VENCER_DE_1081_A_1440_MERCADO',
'CRED_A_VENCER_DE_1441_A_1800_MERCADO']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [122]:
#(Crédito a vencer entre 1081 e 1800 dias)/(Total de crédito a vencer)
df10 = df10.withColumn(f'{modality}_RZ_AVCR_1081_1800_POR_AVCR',round((reduce(add, [F.col(x) for x in lista_1080_1800])/F.col(f'{modality}_AVCR')).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [123]:
#(Crédito a vencer entre 1081 e 1800 dias)/(Total de crédito a vencer)
df10 = df10.withColumn(f'{modality}_RZ_AVCR_1081_1800_POR_AVCR_MERC',round((reduce(add, [F.col(x) for x in lista_1080_1800_mercado])/F.col(f'{modality}_AVCR_MERC')).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [124]:
lista_acima_1800 = ['CRED_A_VENCER_DE_1801_A_5400_TOTAL','CRED_A_VENCER_ACIMA_5400_TOTAL']

lista_acima_1800_mercado = ['CRED_A_VENCER_DE_1801_A_5400_MERCADO','CRED_A_VENCER_ACIMA_5400_MERCADO']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [125]:
#(Crédito a vencer acima de 5400)/(Total de crédito a vencer)
df10 = df10.withColumn(f'{modality}_RZ_AVCR_ACI1800_POR_AVCR',round((reduce(add, [F.col(x) for x in lista_acima_1800])/F.col(f'{modality}_AVCR')).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [126]:
#(Crédito a vencer acima de 5400)/(Total de crédito a vencer)
df10 = df10.withColumn(f'{modality}_RZ_AVCR_ACI1800_POR_AVCR_MERC',round((reduce(add, [F.col(x) for x in lista_acima_1800_mercado])/F.col(f'{modality}_AVCR_MERC')).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

AVCR_ATE90 & AVCR_ATE90_MERC & AVCR_ATE90_RZMO Á AVCR_ACI1800 & AVCR_ACI1800_MERC & AVCR_ACI1800_RZMO

In [127]:
#Crédito a vencer até 90 dias
df10 = df10.withColumn(f'{modality}_AVCR_ATE90',round(reduce(add, [F.col(x) for x in lista_vencer_3meses]).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [128]:
#Crédito a vencer até 90 dias
df10 = df10.withColumn(f'{modality}_AVCR_ATE90_MERC',round(reduce(add, [F.col(x) for x in lista_vencer_3meses_mercado]).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [129]:
#Crédito a vencer até 90 dias - PONDERADO
df10 = df10.withColumn(f'{modality}_AVCR_ATE90_RZMO',round((F.col(f'{modality}_AVCR_ATE90_MERC')/F.col(f'{modality}_SALDO_TOTAL_TT_OMNI')).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [130]:
#Crédito a vencer entre 91 e 180 dias
df10 = df10.withColumn(f'{modality}_AVCR_91_180',round((df10.CRED_A_VENCER_DE_91_A_180_TOTAL).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [131]:
#Crédito a vencer entre 91 e 180 dias
df10 = df10.withColumn(f'{modality}_AVCR_91_180_MERC',round((df10.CRED_A_VENCER_DE_91_A_180_MERCADO).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [132]:
#Crédito a vencer entre 91 e 180 dias - PONDERADO
df10 = df10.withColumn(f'{modality}_AVCR_91_180_RZMO',round((F.col(f'{modality}_AVCR_91_180_MERC')/F.col(f'{modality}_SALDO_TOTAL_TT_OMNI')).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [133]:
#Crédito a vencer entre 181 e 360 dias
df10 = df10.withColumn(f'{modality}_AVCR_181_360',round((df10.CRED_A_VENCER_DE_181_A_360_TOTAL).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [134]:
#Crédito a vencer entre 181 e 360 dias
df10 = df10.withColumn(f'{modality}_AVCR_181_360_MERC',round((df10.CRED_A_VENCER_DE_181_A_360_MERCADO).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [135]:
#Crédito a vencer entre 181 e 360 dias - PONDERADO
df10 = df10.withColumn(f'{modality}_AVCR_181_360_RZMO',round((F.col(f'{modality}_AVCR_181_360_MERC')/F.col(f'{modality}_SALDO_TOTAL_TT_OMNI')).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [136]:
#Crédito a vencer entre 361 e 720 dias
df10 = df10.withColumn(f'{modality}_AVCR_361_720',round((df10.CRED_A_VENCER_DE_361_A_720_TOTAL).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [137]:
#Crédito a vencer entre 361 e 720 dias
df10 = df10.withColumn(f'{modality}_AVCR_361_720_MERC',round((df10.CRED_A_VENCER_DE_361_A_720_MERCADO).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [138]:
#Crédito a vencer entre 361 e 720 dias - PONDERADO
df10 = df10.withColumn(f'{modality}_AVCR_361_720_RZMO',round((F.col(f'{modality}_AVCR_361_720_MERC')/F.col(f'{modality}_SALDO_TOTAL_TT_OMNI')).cast(DoubleType()),2))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [139]:
#Crédito a vencer entre 721 e 1080 dias
df10 = df10.withColumn(f'{modality}_AVCR_721_1080',round((df10.CRED_A_VENCER_DE_721_A_1080_TOTAL).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [140]:
#Crédito a vencer entre 721 e 1080 dias
df10 = df10.withColumn(f'{modality}_AVCR_721_1080_MERC',round((df10.CRED_A_VENCER_DE_721_A_1080_MERCADO).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [141]:
#Crédito a vencer entre 721 e 1080 dias - PONDERADO
df10 = df10.withColumn(f'{modality}_AVCR_721_1080_RZMO',round((F.col(f'{modality}_AVCR_721_1080_MERC')/F.col(f'{modality}_SALDO_TOTAL_TT_OMNI')).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [142]:
#Crédito a vencer entre 1081 e 1800 dias
df10 = df10.withColumn(f'{modality}_AVCR_1081_1800',round((reduce(add, [F.col(x) for x in lista_1080_1800])).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [143]:
#Crédito a vencer entre 1081 e 1800 dias
df10 = df10.withColumn(f'{modality}_AVCR_1081_1800_MERC',round((reduce(add, [F.col(x) for x in lista_1080_1800_mercado])).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [144]:
#Crédito a vencer entre 1081 e 1800 dias - PONDERADO
df10 = df10.withColumn(f'{modality}_AVCR_1081_1800_RZMO',round(((F.col(f'{modality}_AVCR_1081_1800_MERC'))/F.col(f'{modality}_SALDO_TOTAL_TT_OMNI')).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [145]:
#Crédito a vencer acima de 1800
df10 = df10.withColumn(f'{modality}_AVCR_ACI1800',round((reduce(add, [F.col(x) for x in lista_acima_1800])).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [146]:
#Crédito a vencer acima de 1800
df10 = df10.withColumn(f'{modality}_AVCR_ACI1800_MERC',round((reduce(add, [F.col(x) for x in lista_acima_1800_mercado])).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [147]:
#Crédito a vencer acima de 1800 - PONDERADO
df10 = df10.withColumn(f'{modality}_AVCR_ACI1800_RZMO',round((F.col(f'{modality}_AVCR_ACI1800_MERC')/F.col(f'{modality}_SALDO_TOTAL_TT_OMNI')).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

RZ_VCD_ATE30_POR_VCD & RZ_VCD_ATE30_POR_VCD_MERC Á RZ_VCD_181_360_POR_VCD & RZ_VCD_181_360_POR_VCD_MERC

In [148]:
lista_vencido_ate_30 = ['CRED_VENCIDOS_DE_1_A_14_TOTAL','CRED_VENCIDOS_DE_15_A_30_TOTAL']

lista_vencido_ate_30_mercado = ['CRED_VENCIDOS_DE_1_A_14_MERCADO','CRED_VENCIDOS_DE_15_A_30_MERCADO']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [149]:
#Crédito vencido até 30 dias/Total crédito vencido
df10 = df10.withColumn(f'{modality}_RZ_VCD_ATE30_POR_VCD',round(((reduce(add, [F.col(x) for x in lista_vencido_ate_30]))/F.col(f'{modality}_VCD')).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [150]:
#Crédito vencido até 30 dias/Total crédito vencido
df10 = df10.withColumn(f'{modality}_RZ_VCD_ATE30_POR_VCD_MERC',round(((reduce(add, [F.col(x) for x in lista_vencido_ate_30_mercado]))/F.col(f'{modality}_VCD_MERC')).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [151]:
#Crédito vencido entre 31 e 60 dias/Total crédito vencido
df10 = df10.withColumn(f'{modality}_RZ_VCD_31_60_POR_VCD',round((df10.CRED_VENCIDOS_DE_31_A_60_TOTAL/F.col(f'{modality}_VCD')).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [152]:
#Crédito vencido entre 31 e 60 dias/Total crédito vencido
df10 = df10.withColumn(f'{modality}_RZ_VCD_31_60_POR_VCD_MERC',round((df10.CRED_VENCIDOS_DE_31_A_60_MERCADO/F.col(f'{modality}_VCD_MERC')).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [153]:
#Crédito vencido entre 61 e 90 dias/Total crédito vencido
df10 = df10.withColumn(f'{modality}_RZ_VCD_61_90_POR_VCD',round((df10.CRED_VENCIDOS_DE_61_A_90_TOTAL/F.col(f'{modality}_VCD')).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [154]:
#Crédito vencido entre 61 e 90 dias/Total crédito vencido
df10 = df10.withColumn(f'{modality}_RZ_VCD_61_90_POR_VCD_MERC',round((df10.CRED_VENCIDOS_DE_61_A_90_MERCADO/F.col(f'{modality}_VCD_MERC')).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [155]:
lista_vencido_91_ate_180 = ['CRED_VENCIDOS_DE_91_A_120_TOTAL','CRED_VENCIDOS_DE_121_A_150_TOTAL','CRED_VENCIDOS_DE_151_A_180_TOTAL']

lista_vencido_91_ate_180_mercado = ['CRED_VENCIDOS_DE_91_A_120_MERCADO','CRED_VENCIDOS_DE_121_A_150_MERCADO','CRED_VENCIDOS_DE_151_A_180_MERCADO']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [156]:
#Crédito vencido entre 91 e 180 dias/Total crédito vencido
df10 = df10.withColumn(f'{modality}_RZ_VCD_91_180_POR_VCD',round(((reduce(add, [F.col(x) for x in lista_vencido_91_ate_180]))/F.col(f'{modality}_VCD')).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [157]:
#Crédito vencido entre 91 e 180 dias/Total crédito vencido
df10 = df10.withColumn(f'{modality}_RZ_VCD_91_180_POR_VCD_MERC',round(((reduce(add, [F.col(x) for x in lista_vencido_91_ate_180_mercado]))/F.col(f'{modality}_VCD_MERC')).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [158]:
lista_vencido_181_ate_360 = ['CRED_VENCIDOS_DE_181_A_240_TOTAL','CRED_VENCIDOS_DE_241_A_300_TOTAL','CRED_VENCIDOS_DE_301_A_360_TOTAL']

lista_vencido_181_ate_360_mercado = ['CRED_VENCIDOS_DE_181_A_240_MERCADO','CRED_VENCIDOS_DE_241_A_300_MERCADO','CRED_VENCIDOS_DE_301_A_360_MERCADO']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [159]:
#Crédito vencido entre 181 e 360 dias/Total crédito vencido
df10 = df10.withColumn(f'{modality}_RZ_VCD_181_360_POR_VCD',round(((reduce(add, [F.col(x) for x in lista_vencido_181_ate_360]))/F.col(f'{modality}_VCD')).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [160]:
#Crédito vencido entre 181 e 360 dias/Total crédito vencido
df10 = df10.withColumn(f'{modality}_RZ_VCD_181_360_POR_VCD_MERC',round(((reduce(add, [F.col(x) for x in lista_vencido_181_ate_360_mercado]))/F.col(f'{modality}_VCD_MERC')).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

VCD_ATE30 & VCD_ATE30_MERC & VCD_ATE30_RZMO Á  VCD_181_360 & VCD_181_360_MERC & VCD_181_360_RZMO

In [161]:
#Crédito vencido até 30 dias
df10 = df10.withColumn(f'{modality}_VCD_ATE30',round((reduce(add, [F.col(x) for x in lista_vencido_ate_30])).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [162]:
#Crédito vencido até 30 dias
df10 = df10.withColumn(f'{modality}_VCD_ATE30_MERC',round((reduce(add, [F.col(x) for x in lista_vencido_ate_30_mercado])).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [163]:
#Crédito vencido até 30 dias - PONDERADO
df10 = df10.withColumn(f'{modality}_VCD_ATE30_RZMO',round((F.col(f'{modality}_VCD_ATE30_MERC')/F.col(f'{modality}_SALDO_TOTAL_TT_OMNI')).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [164]:
#Crédito vencido entre 31 e 60 dias
df10 = df10.withColumn(f'{modality}_VCD_31_60',round((df10.CRED_VENCIDOS_DE_31_A_60_TOTAL).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [165]:
#Crédito vencido entre 31 e 60 dias
df10 = df10.withColumn(f'{modality}_VCD_31_60_MERC',round((df10.CRED_VENCIDOS_DE_31_A_60_MERCADO).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [166]:
#Crédito vencido entre 31 e 60 dias - PONDERADO
df10 = df10.withColumn(f'{modality}_VCD_31_60_RZMO',round((F.col(f'{modality}_VCD_31_60_MERC')/F.col(f'{modality}_SALDO_TOTAL_TT_OMNI')).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [167]:
#Crédito vencido entre 61 e 90 dias
df10 = df10.withColumn(f'{modality}_VCD_61_90',round((df10.CRED_VENCIDOS_DE_61_A_90_TOTAL).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [168]:
#Crédito vencido entre 61 e 90 dias
df10 = df10.withColumn(f'{modality}_VCD_61_90_MERC',round((df10.CRED_VENCIDOS_DE_61_A_90_MERCADO).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [169]:
#Crédito vencido entre 61 e 90 dias - PONDERADO
df10 = df10.withColumn(f'{modality}_VCD_61_90_RZMO',round((F.col(f'{modality}_VCD_61_90_MERC')/F.col(f'{modality}_SALDO_TOTAL_TT_OMNI')).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [170]:
#Crédito vencido entre 91 e 180 dias
df10 = df10.withColumn(f'{modality}_VCD_91_180',round((reduce(add, [F.col(x) for x in lista_vencido_91_ate_180])).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [171]:
#Crédito vencido entre 91 e 180 dias
df10 = df10.withColumn(f'{modality}_VCD_91_180_MERC',round((reduce(add, [F.col(x) for x in lista_vencido_91_ate_180_mercado])).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [172]:
#Crédito vencido entre 91 e 180 dias - PONDERADO
df10 = df10.withColumn(f'{modality}_VCD_91_180_RZMO',round((F.col(f'{modality}_VCD_91_180_MERC')/F.col(f'{modality}_SALDO_TOTAL_TT_OMNI')).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [173]:
#Crédito vencido entre 181 e 360 dias
df10 = df10.withColumn(f'{modality}_VCD_181_360',round((reduce(add, [F.col(x) for x in lista_vencido_181_ate_360])).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [174]:
#Crédito vencido entre 181 e 360 dias
df10 = df10.withColumn(f'{modality}_VCD_181_360_MERC',round((reduce(add, [F.col(x) for x in lista_vencido_181_ate_360_mercado])).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [175]:
#Crédito vencido entre 181 e 360 dias - PONDERADO
df10 = df10.withColumn(f'{modality}_VCD_181_360_RZMO',round((F.col(f'{modality}_VCD_181_360_MERC')/F.col(f'{modality}_SALDO_TOTAL_TT_OMNI')).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [176]:
lista_divida_total = ["CRED_A_VENCER_ATE_30_TOTAL",
"CRED_A_VENCER_DE_31_A_60_TOTAL",
"CRED_A_VENCER_DE_61_A_90_TOTAL",
"CRED_A_VENCER_DE_91_A_180_TOTAL",
"CRED_A_VENCER_DE_181_A_360_TOTAL",
"CRED_A_VENCER_DE_361_A_720_TOTAL",
"CRED_A_VENCER_DE_721_A_1080_TOTAL",
"CRED_A_VENCER_DE_1081_A_1440_TOTAL",
"CRED_A_VENCER_DE_1441_A_1800_TOTAL",
"CRED_A_VENCER_DE_1801_A_5400_TOTAL",
"CRED_A_VENCER_ACIMA_5400_TOTAL",
"CRED_A_VENCER_PRAZO_INDETERM_TOTAL",
"CRED_VENCIDOS_DE_1_A_14_TOTAL",
"CRED_VENCIDOS_DE_15_A_30_TOTAL",
"CRED_VENCIDOS_DE_31_A_60_TOTAL",
"CRED_VENCIDOS_DE_61_A_90_TOTAL",
"CRED_VENCIDOS_DE_91_A_120_TOTAL",
"CRED_VENCIDOS_DE_121_A_150_TOTAL",
"CRED_VENCIDOS_DE_151_A_180_TOTAL",
"CRED_VENCIDOS_DE_181_A_240_TOTAL",
"CRED_VENCIDOS_DE_241_A_300_TOTAL",
"CRED_VENCIDOS_DE_301_A_360_TOTAL",
"CRED_VENCIDOS_DE_361_A_540_TOTAL",
"CRED_VENCIDOS_ACIMA_540_TOTAL",
"CRED_BAIX_PREJ_ATE_12_M_TOTAL",
"CRED_BAIX_PREJ_MAIS_12_A_48_M_TOTAL",
"CRED_BAIX_PREJ_MAIS_48_M_TOTAL"]

lista_divida_total_mercado = ["CRED_A_VENCER_ATE_30_MERCADO",
"CRED_A_VENCER_DE_31_A_60_MERCADO",
"CRED_A_VENCER_DE_61_A_90_MERCADO",
"CRED_A_VENCER_DE_91_A_180_MERCADO",
"CRED_A_VENCER_DE_181_A_360_MERCADO",
"CRED_A_VENCER_DE_361_A_720_MERCADO",
"CRED_A_VENCER_DE_721_A_1080_MERCADO",
"CRED_A_VENCER_DE_1081_A_1440_MERCADO",
"CRED_A_VENCER_DE_1441_A_1800_MERCADO",
"CRED_A_VENCER_DE_1801_A_5400_MERCADO",
"CRED_A_VENCER_ACIMA_5400_MERCADO",
"CRED_A_VENCER_PRAZO_INDETERM_MERCADO",
"CRED_VENCIDOS_DE_1_A_14_MERCADO",
"CRED_VENCIDOS_DE_15_A_30_MERCADO",
"CRED_VENCIDOS_DE_31_A_60_MERCADO",
"CRED_VENCIDOS_DE_61_A_90_MERCADO",
"CRED_VENCIDOS_DE_91_A_120_MERCADO",
"CRED_VENCIDOS_DE_121_A_150_MERCADO",
"CRED_VENCIDOS_DE_151_A_180_MERCADO",
"CRED_VENCIDOS_DE_181_A_240_MERCADO",
"CRED_VENCIDOS_DE_241_A_300_MERCADO",
"CRED_VENCIDOS_DE_301_A_360_MERCADO",
"CRED_VENCIDOS_DE_361_A_540_MERCADO",
"CRED_VENCIDOS_ACIMA_540_MERCADO",
"CRED_BAIX_PREJ_ATE_12_M_MERCADO",
"CRED_BAIX_PREJ_MAIS_12_A_48_M_MERCADO",
"CRED_BAIX_PREJ_MAIS_48_M_MERCADO"]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SALDO_TOTAL & SALDO_TOTAL_MERC & SALDO_TOTAL_RZMO

In [177]:
df10 = df10.withColumn(f'{modality}_SALDO_TOTAL',round((reduce(add, [F.col(x) for x in lista_divida_total])).cast(DoubleType()),4))#listada no início do código

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [178]:
df10 = df10.withColumn(f'{modality}_SALDO_TOTAL_MERC',round((reduce(add, [F.col(x) for x in lista_divida_total_mercado])).cast(DoubleType()),4))#listada no início do código

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [179]:
df10 = df10.withColumn(f'{modality}_SALDO_TOTAL_RZMO',round((F.col(f'{modality}_SALDO_TOTAL_MERC')/F.col(f'{modality}_SALDO_TOTAL_TT_OMNI')).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [180]:
lista_vencido_prejuizo = ["CRED_VENCIDOS_DE_1_A_14_TOTAL",
"CRED_VENCIDOS_DE_15_A_30_TOTAL",
"CRED_VENCIDOS_DE_31_A_60_TOTAL",
"CRED_VENCIDOS_DE_61_A_90_TOTAL",
"CRED_VENCIDOS_DE_91_A_120_TOTAL",
"CRED_VENCIDOS_DE_121_A_150_TOTAL",
"CRED_VENCIDOS_DE_151_A_180_TOTAL",
"CRED_VENCIDOS_DE_181_A_240_TOTAL",
"CRED_VENCIDOS_DE_241_A_300_TOTAL",
"CRED_VENCIDOS_DE_301_A_360_TOTAL",
"CRED_VENCIDOS_DE_361_A_540_TOTAL",
"CRED_VENCIDOS_ACIMA_540_TOTAL",
"CRED_BAIX_PREJ_ATE_12_M_TOTAL",
"CRED_BAIX_PREJ_MAIS_12_A_48_M_TOTAL",
"CRED_BAIX_PREJ_MAIS_48_M_TOTAL"]

lista_vencido_prejuizo_mercado = ["CRED_VENCIDOS_DE_1_A_14_MERCADO",
"CRED_VENCIDOS_DE_15_A_30_MERCADO",
"CRED_VENCIDOS_DE_31_A_60_MERCADO",
"CRED_VENCIDOS_DE_61_A_90_MERCADO",
"CRED_VENCIDOS_DE_91_A_120_MERCADO",
"CRED_VENCIDOS_DE_121_A_150_MERCADO",
"CRED_VENCIDOS_DE_151_A_180_MERCADO",
"CRED_VENCIDOS_DE_181_A_240_MERCADO",
"CRED_VENCIDOS_DE_241_A_300_MERCADO",
"CRED_VENCIDOS_DE_301_A_360_MERCADO",
"CRED_VENCIDOS_DE_361_A_540_MERCADO",
"CRED_VENCIDOS_ACIMA_540_MERCADO",
"CRED_BAIX_PREJ_ATE_12_M_MERCADO",
"CRED_BAIX_PREJ_MAIS_12_A_48_M_MERCADO",
"CRED_BAIX_PREJ_MAIS_48_M_MERCADO"]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

PREJ & PREJ_MERC & PREJ_RZMO

In [181]:
df10 = df10.withColumn(f'{modality}_PREJ',round((reduce(add, [F.col(x) for x in lista_vencido_prejuizo])).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [182]:
df10 = df10.withColumn(f'{modality}_PREJ_MERC',round((reduce(add, [F.col(x) for x in lista_vencido_prejuizo_mercado])).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [183]:
df10 = df10.withColumn(f'{modality}_PREJ_RZMO',round((F.col(f'{modality}_PREJ_MERC')/F.col(f'{modality}_SALDO_TOTAL_TT_OMNI')).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

VAR_MAIOR_ATRASO_M0_M1 & VAR_SALDO_M0_M1 & VAR_VCD_PREJ_M0_M1

RZ_SALDO_M1_M0 & RZ_VCD_M1_M0 & RZ_AVCR_M1_M0 & RZ_PREJ_M1_M0

In [184]:
#Variação do maior atraso entre o mês atual e o mês anterior
from pyspark.sql.window import Window

w_chaves = Window.partitionBy().orderBy("CD","M0")

df10 = df10.withColumn("prev_key_dtbase",F.lag(df10.M0).over(w_chaves))
df10 = df10.withColumn("prev_key_cd",F.lag(df10.CD).over(w_chaves))
df10 = df10.withColumn(f"prev_{modality}_MAIOR_ATRASO_VCD",F.lag(F.col(f'{modality}_MAIOR_ATRASO_VCD')).over(w_chaves))
df10 = df10.withColumn(f"prev_{modality}_RZ_SALDO_TT_ANTR",F.lag(F.col(f'{modality}_SALDO_TOTAL')).over(w_chaves))
df10 = df10.withColumn(f"prev_{modality}_RZ_SALDO_ATRS_PREJ_MES_ANTR",F.lag(F.col(f'{modality}_PREJ')).over(w_chaves))
df10 = df10.withColumn(f"prev_{modality}_RZ_LMT_DSPN_MES_ANTR",F.lag(F.col(f'{modality}_LMT_DISPONIVEL')).over(w_chaves))
df10 = df10.withColumn(f"prev_{modality}_RZ_LMT_TT_MES_ANTR",F.lag(F.col(f'{modality}_LMT_TOTAL')).over(w_chaves))

df10 = df10.withColumn(f"prev_{modality}_SALDO_TOTAL_TT_TOTAL_ANTR",F.lag(F.col(f'{modality}_SALDO_TOTAL_TT_TOTAL')).over(w_chaves))
df10 = df10.withColumn(f"prev_{modality}_SALDO_VCD_TOTAL_TT_TOTAL_ANTR",F.lag(F.col(f'{modality}_SALDO_VCD_TOTAL_TT_TOTAL')).over(w_chaves))
df10 = df10.withColumn(f"prev_{modality}_SALDO_AVCR_TOTAL_TT_TOTAL_ANTR",F.lag(F.col(f'{modality}_SALDO_AVCR_TOTAL_TT_TOTAL')).over(w_chaves))
df10 = df10.withColumn(f"prev_{modality}_SALDO_PREJUIZO_TOTAL_TT_TOTAL_ANTR",F.lag(F.col(f'{modality}_SALDO_PREJUIZO_TOTAL_TT_TOTAL')).over(w_chaves))

df10 = df10.withColumn(f"{modality}_VAR_MAIOR_ATRASO_M0_M1", when(F.isnull(F.col(f'{modality}_MAIOR_ATRASO_VCD') - F.col(f'prev_{modality}_MAIOR_ATRASO_VCD')), 0) \
                           .when((df10.prev_key_cd == df10.CD) & \
                                 (df10.prev_key_dtbase == (df10.M1)), \
                                  F.col(f'{modality}_MAIOR_ATRASO_VCD') - F.col(f'prev_{modality}_MAIOR_ATRASO_VCD'))
                         .otherwise(0.0))

df10 = df10.withColumn(f"{modality}_VAR_SALDO_M0_M1", when(F.isnull(F.col(f'{modality}_SALDO_TOTAL') - F.col(f'prev_{modality}_RZ_SALDO_TT_ANTR')), 0) \
                           .when((df10.prev_key_cd == df10.CD) & \
                                 (df10.prev_key_dtbase == (df10.M1)), \
                                  F.col(f'{modality}_SALDO_TOTAL') - F.col(f'prev_{modality}_RZ_SALDO_TT_ANTR'))
                         .otherwise(0.0))
                       
df10 = df10.withColumn(f"{modality}_VAR_VCD_PREJ_M0_M1", when(F.isnull(F.col(f'{modality}_PREJ') - F.col(f'prev_{modality}_RZ_SALDO_ATRS_PREJ_MES_ANTR')), 0) \
                           .when((df10.prev_key_cd == df10.CD) & \
                                 (df10.prev_key_dtbase == (df10.M1)), \
                                  F.col(f'{modality}_PREJ') - F.col(f'prev_{modality}_RZ_SALDO_ATRS_PREJ_MES_ANTR'))
                         .otherwise(0.0))
#NOVOS                         
df10 = df10.withColumn(f"{modality}_VAR_LMT_DSPN_M0_M1", when(F.isnull(F.col(f'{modality}_LMT_DISPONIVEL') - F.col(f'prev_{modality}_RZ_LMT_DSPN_MES_ANTR')), 0) \
                           .when((df10.prev_key_cd == df10.CD) & \
                                 (df10.prev_key_dtbase == (df10.M1)), \
                                  F.col(f'{modality}_LMT_DISPONIVEL') - F.col(f'prev_{modality}_RZ_LMT_DSPN_MES_ANTR'))
                         .otherwise(0.0))

df10 = df10.withColumn(f"{modality}_VAR_LMT_TT_M0_M1", when(F.isnull(F.col(f'{modality}_LMT_TOTAL') - F.col(f'prev_{modality}_RZ_LMT_TT_MES_ANTR')), 0) \
                           .when((df10.prev_key_cd == df10.CD) & \
                                 (df10.prev_key_dtbase == (df10.M1)), \
                                  F.col(f'{modality}_LMT_TOTAL') - F.col(f'prev_{modality}_RZ_LMT_TT_MES_ANTR'))
                         .otherwise(0.0))
#NOVOS SALDO
df10 = df10.withColumn(f"{modality}_RZ_SALDO_M1_M0", when(F.isnull(F.col(f'{modality}_SALDO_TOTAL_TT_TOTAL') - F.col(f'prev_{modality}_SALDO_TOTAL_TT_TOTAL_ANTR')), 0) \
                           .when((df10.prev_key_cd == df10.CD) & \
                                 (df10.prev_key_dtbase == (df10.M1)), \
                                  F.col(f'{modality}_SALDO_TOTAL_TT_TOTAL') - F.col(f'prev_{modality}_SALDO_TOTAL_TT_TOTAL_ANTR'))
                         .otherwise(0.0))

df10 = df10.withColumn(f"{modality}_RZ_VCD_M1_M0", when(F.isnull(F.col(f'{modality}_SALDO_VCD_TOTAL_TT_TOTAL') - F.col(f'prev_{modality}_SALDO_VCD_TOTAL_TT_TOTAL_ANTR')), 0) \
                           .when((df10.prev_key_cd == df10.CD) & \
                                 (df10.prev_key_dtbase == (df10.M1)), \
                                  F.col(f'{modality}_SALDO_VCD_TOTAL_TT_TOTAL') - F.col(f'prev_{modality}_SALDO_VCD_TOTAL_TT_TOTAL_ANTR'))
                         .otherwise(0.0))

df10 = df10.withColumn(f"{modality}_RZ_AVCR_M1_M0", when(F.isnull(F.col(f'{modality}_SALDO_AVCR_TOTAL_TT_TOTAL') - F.col(f'prev_{modality}_SALDO_AVCR_TOTAL_TT_TOTAL_ANTR')), 0) \
                           .when((df10.prev_key_cd == df10.CD) & \
                                 (df10.prev_key_dtbase == (df10.M1)), \
                                  F.col(f'{modality}_SALDO_AVCR_TOTAL_TT_TOTAL') - F.col(f'prev_{modality}_SALDO_AVCR_TOTAL_TT_TOTAL_ANTR'))
                         .otherwise(0.0))

df10 = df10.withColumn(f"{modality}_RZ_PREJ_M1_M0", when(F.isnull(F.col(f'{modality}_SALDO_PREJUIZO_TOTAL_TT_TOTAL') - F.col(f'prev_{modality}_SALDO_PREJUIZO_TOTAL_TT_TOTAL_ANTR')), 0) \
                           .when((df10.prev_key_cd == df10.CD) & \
                                 (df10.prev_key_dtbase == (df10.M1)), \
                                  F.col(f'{modality}_SALDO_PREJUIZO_TOTAL_TT_TOTAL') - F.col(f'prev_{modality}_SALDO_PREJUIZO_TOTAL_TT_TOTAL_ANTR'))
                         .otherwise(0.0))



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

VAR_MAIOR_ATRASO_M0_M1_MERC & VAR_SALDO_M0_M1_MERC & VAR_VCD_PREJ_M0_M1_MERC

RZ_SALDO_M1_M0_MERC & RZ_VCD_M1_M0_MERC & RZ_AVCR_M1_M0_MERC & RZ_PREJ_M1_M0_MERC

In [185]:
df10 = df10.withColumn("prev_key_dtbase",F.lag(df10.M0).over(w_chaves))
df10 = df10.withColumn("prev_key_cd",F.lag(df10.CD).over(w_chaves))
df10 = df10.withColumn(f"prev_{modality}_MAIOR_ATRASO_VCD_MERC",F.lag(F.col(f'{modality}_MAIOR_ATRASO_VCD_MERC')).over(w_chaves))
df10 = df10.withColumn(f"prev_{modality}_RZ_SALDO_TT_ANTR_MERC",F.lag(F.col(f'{modality}_SALDO_TOTAL_MERC')).over(w_chaves))
df10 = df10.withColumn(f"prev_{modality}_RZ_SALDO_ATRS_PREJ_MES_ANTR_MERC",F.lag(F.col(f'{modality}_PREJ_MERC')).over(w_chaves))
df10 = df10.withColumn(f"prev_{modality}_RZ_LMT_DSPN_MES_ANTR_MERC",F.lag(F.col(f'{modality}_LMT_DISPONIVEL_MERC')).over(w_chaves))
df10 = df10.withColumn(f"prev_{modality}_RZ_LMT_TT_MES_ANTR_MERC",F.lag(F.col(f'{modality}_LMT_TOTAL_MERC')).over(w_chaves))

df10 = df10.withColumn(f"prev_{modality}_SALDO_TOTAL_TT_TOTAL_ANTR_MERC",F.lag(F.col(f'{modality}_SALDO_TOTAL_TT_MERCADO')).over(w_chaves))
df10 = df10.withColumn(f"prev_{modality}_SALDO_VCD_TOTAL_TT_TOTAL_ANTR_MERC",F.lag(F.col(f'{modality}_SALDO_VCD_TOTAL_TT_MERCADO')).over(w_chaves))
df10 = df10.withColumn(f"prev_{modality}_SALDO_AVCR_TOTAL_TT_TOTAL_ANTR_MERC",F.lag(F.col(f'{modality}_SALDO_AVCR_TOTAL_TT_MERCADO')).over(w_chaves))
df10 = df10.withColumn(f"prev_{modality}_SALDO_PREJUIZO_TOTAL_TT_TOTAL_ANTR_MERC",F.lag(F.col(f'{modality}_SALDO_PREJUIZO_TOTAL_TT_MERCADO')).over(w_chaves))


df10 = df10.withColumn(f"{modality}_VAR_MAIOR_ATRASO_M0_M1_MERC", when(F.isnull(F.col(f'{modality}_MAIOR_ATRASO_VCD_MERC') - F.col(f'prev_{modality}_MAIOR_ATRASO_VCD_MERC')), 0) \
                           .when((df10.prev_key_cd == df10.CD) & \
                                 (df10.prev_key_dtbase == (df10.M1)), \
                                  F.col(f'{modality}_MAIOR_ATRASO_VCD_MERC') - F.col(f'prev_{modality}_MAIOR_ATRASO_VCD_MERC'))
                         .otherwise(0.0))

df10 = df10.withColumn(f"{modality}_VAR_SALDO_M0_M1_MERC", when(F.isnull(F.col(f'{modality}_SALDO_TOTAL_MERC') - F.col(f'prev_{modality}_RZ_SALDO_TT_ANTR_MERC')), 0) \
                           .when((df10.prev_key_cd == df10.CD) & \
                                 (df10.prev_key_dtbase == (df10.M1)), \
                                  F.col(f'{modality}_SALDO_TOTAL_MERC') - F.col(f'prev_{modality}_RZ_SALDO_TT_ANTR_MERC'))
                         .otherwise(0.0))
                       
df10 = df10.withColumn(f"{modality}_VAR_VCD_PREJ_M0_M1_MERC", when(F.isnull(F.col(f'{modality}_PREJ_MERC') - F.col(f'prev_{modality}_RZ_SALDO_ATRS_PREJ_MES_ANTR_MERC')), 0) \
                           .when((df10.prev_key_cd == df10.CD) & \
                                 (df10.prev_key_dtbase == (df10.M1)), \
                                  F.col(f'{modality}_PREJ_MERC') - F.col(f'prev_{modality}_RZ_SALDO_ATRS_PREJ_MES_ANTR_MERC'))
                         .otherwise(0.0))
                        
df10 = df10.withColumn(f"{modality}_VAR_LMT_DSPN_M0_M1_MERC", when(F.isnull(F.col(f'{modality}_LMT_DISPONIVEL_MERC') - F.col(f'prev_{modality}_RZ_LMT_DSPN_MES_ANTR_MERC')), 0) \
                           .when((df10.prev_key_cd == df10.CD) & \
                                 (df10.prev_key_dtbase == (df10.M1)), \
                                  F.col(f'{modality}_LMT_DISPONIVEL_MERC') - F.col(f'prev_{modality}_RZ_LMT_DSPN_MES_ANTR_MERC'))
                         .otherwise(0.0))

df10 = df10.withColumn(f"{modality}_VAR_LMT_TT_M0_M1_MERC", when(F.isnull(F.col(f'{modality}_LMT_TOTAL_MERC') - F.col(f'prev_{modality}_RZ_LMT_TT_MES_ANTR_MERC')), 0) \
                           .when((df10.prev_key_cd == df10.CD) & \
                                 (df10.prev_key_dtbase == (df10.M1)), \
                                  F.col(f'{modality}_LMT_TOTAL_MERC') - F.col(f'prev_{modality}_RZ_LMT_TT_MES_ANTR_MERC'))
                         .otherwise(0.0))

df10 = df10.withColumn(f"{modality}_RZ_SALDO_M1_M0_MERC", when(F.isnull(F.col(f'{modality}_SALDO_TOTAL_TT_MERCADO') - F.col(f'prev_{modality}_SALDO_TOTAL_TT_TOTAL_ANTR_MERC')), 0) \
                           .when((df10.prev_key_cd == df10.CD) & \
                                 (df10.prev_key_dtbase == (df10.M1)), \
                                  F.col(f'{modality}_SALDO_TOTAL_TT_MERCADO') - F.col(f'prev_{modality}_SALDO_TOTAL_TT_TOTAL_ANTR_MERC'))
                         .otherwise(0.0))

df10 = df10.withColumn(f"{modality}_RZ_VCD_M1_M0_MERC", when(F.isnull(F.col(f'{modality}_SALDO_VCD_TOTAL_TT_MERCADO') - F.col(f'prev_{modality}_SALDO_VCD_TOTAL_TT_TOTAL_ANTR_MERC')), 0) \
                           .when((df10.prev_key_cd == df10.CD) & \
                                 (df10.prev_key_dtbase == (df10.M1)), \
                                  F.col(f'{modality}_SALDO_VCD_TOTAL_TT_MERCADO') - F.col(f'prev_{modality}_SALDO_VCD_TOTAL_TT_TOTAL_ANTR_MERC'))
                         .otherwise(0.0))

df10 = df10.withColumn(f"{modality}_RZ_AVCR_M1_M0_MERC", when(F.isnull(F.col(f'{modality}_SALDO_AVCR_TOTAL_TT_MERCADO') - F.col(f'prev_{modality}_SALDO_AVCR_TOTAL_TT_TOTAL_ANTR_MERC')), 0) \
                           .when((df10.prev_key_cd == df10.CD) & \
                                 (df10.prev_key_dtbase == (df10.M1)), \
                                  F.col(f'{modality}_SALDO_AVCR_TOTAL_TT_MERCADO') - F.col(f'prev_{modality}_SALDO_AVCR_TOTAL_TT_TOTAL_ANTR_MERC'))
                         .otherwise(0.0))

df10 = df10.withColumn(f"{modality}_RZ_PREJ_M1_M0_MERC", when(F.isnull(F.col(f'{modality}_SALDO_PREJUIZO_TOTAL_TT_MERCADO') - F.col(f'prev_{modality}_SALDO_PREJUIZO_TOTAL_TT_TOTAL_ANTR_MERC')), 0) \
                           .when((df10.prev_key_cd == df10.CD) & \
                                 (df10.prev_key_dtbase == (df10.M1)), \
                                  F.col(f'{modality}_SALDO_PREJUIZO_TOTAL_TT_MERCADO') - F.col(f'prev_{modality}_SALDO_PREJUIZO_TOTAL_TT_TOTAL_ANTR_MERC'))
                         .otherwise(0.0))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [186]:
for column in df10.columns:
    if column.endswith('M1_M0') or column.endswith('M1_M0_MERC') or column.endswith('M1_M0_RZMO'):
        df10 = df10.withColumn(column, F.round(column, 4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

PERC_LIMITE_DISP_3M & PERC_LIMITE_DISP_3M_MERC

In [187]:
if modality in lista_limite_total_modalidade:

    df_limite_disponivel_3_M0 = df10.select("CD","M0",f'{modality}_LMT_DISPONIVEL')
    df_limite_disponivel_3_M0 = df_limite_disponivel_3_M0.withColumnRenamed(f'{modality}_LMT_DISPONIVEL', f'{modality}_LMT_DISPONIVEL_M1')

    df_limite_disponivel_3_M1 = df10.select("CD","M0",f'{modality}_LMT_DISPONIVEL')
    df_limite_disponivel_3_M1 = df_limite_disponivel_3_M1.withColumnRenamed('M0', 'M1')
    df_limite_disponivel_3_M1 = df_limite_disponivel_3_M1.withColumnRenamed(f'{modality}_LMT_DISPONIVEL', f'{modality}_LMT_DISPONIVEL_M2')

    df_limite_disponivel_3_M2 = df10.select("CD","M0",f'{modality}_LMT_DISPONIVEL')
    df_limite_disponivel_3_M2 = df_limite_disponivel_3_M2.withColumnRenamed('M0', 'M2')
    df_limite_disponivel_3_M2 = df_limite_disponivel_3_M2.withColumnRenamed(f'{modality}_LMT_DISPONIVEL', f'{modality}_LMT_DISPONIVEL_M3')

    df10 = df10.join(df_limite_disponivel_3_M0,["M0","CD"],"left")# join entre M0 e M0
    df10 = df10.join(df_limite_disponivel_3_M1,["M1","CD"],"left")# join entre M0 e M1
    df10 = df10.join(df_limite_disponivel_3_M2,["M2","CD"],"left")# join entre M0 e M2

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [188]:
if modality in lista_limite_total_modalidade:    
    df_limite_disponivel_3_MERC_M0 = df10.select("CD","M0",f'{modality}_LMT_DISPONIVEL_MERC')
    df_limite_disponivel_3_MERC_M0 = df_limite_disponivel_3_MERC_M0.withColumnRenamed(f'{modality}_LMT_DISPONIVEL_MERC', f'{modality}_LMT_DISPONIVEL_MERC_M1')

    df_limite_disponivel_3_MERC_M1 = df10.select("CD","M0",f'{modality}_LMT_DISPONIVEL_MERC')
    df_limite_disponivel_3_MERC_M1 = df_limite_disponivel_3_MERC_M1.withColumnRenamed('M0', 'M1')
    df_limite_disponivel_3_MERC_M1 = df_limite_disponivel_3_MERC_M1.withColumnRenamed(f'{modality}_LMT_DISPONIVEL_MERC', f'{modality}_LMT_DISPONIVEL_MERC_M2')

    df_limite_disponivel_3_MERC_M2 = df10.select("CD","M0",f'{modality}_LMT_DISPONIVEL_MERC')
    df_limite_disponivel_3_MERC_M2 = df_limite_disponivel_3_MERC_M2.withColumnRenamed('M0', 'M2')
    df_limite_disponivel_3_MERC_M2 = df_limite_disponivel_3_MERC_M2.withColumnRenamed(f'{modality}_LMT_DISPONIVEL_MERC', f'{modality}_LMT_DISPONIVEL_MERC_M3')

    df10 = df10.join(df_limite_disponivel_3_MERC_M0,["M0","CD"],"left")# join entre M0 e M0
    df10 = df10.join(df_limite_disponivel_3_MERC_M1,["M1","CD"],"left")# join entre M0 e M1
    df10 = df10.join(df_limite_disponivel_3_MERC_M2,["M2","CD"],"left")# join entre M0 e M2

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [189]:
if modality in lista_limite_total_modalidade:
    lista_limite_disponivel_3 = [f'{modality}_LMT_DISPONIVEL_M1', f'{modality}_LMT_DISPONIVEL_M2', f'{modality}_LMT_DISPONIVEL_M3']

    lista_limite_disponivel_3_MERC = [f'{modality}_LMT_DISPONIVEL_MERC_M1', f'{modality}_LMT_DISPONIVEL_MERC_M2', f'{modality}_LMT_DISPONIVEL_MERC_M3']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [190]:
if modality in lista_limite_total_modalidade:
    df10 = df10.withColumn(f'{modality}_PERC_LIMITE_DISP_3M', F.round((reduce(add, [F.col(x) for x in lista_limite_disponivel_3])/3).cast(DoubleType()),4))
    df10 = df10.withColumn(f'{modality}_PERC_LIMITE_DISP_3M_MERC', F.round((reduce(add, [F.col(x) for x in lista_limite_disponivel_3_MERC])/3).cast(DoubleType()),4))
    df10 = df10.withColumn(f'{modality}_PERC_LIMITE_DISP_3M_V2', F.round((reduce(add, [F.col(x) for x in lista_limite_disponivel_3])).cast(DoubleType()),4))
    df10 = df10.withColumn(f'{modality}_PERC_LIMITE_DISP_3M_MERC_V2', F.round((reduce(add, [F.col(x) for x in lista_limite_disponivel_3_MERC])).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

VAR_SALDO_M0_M1_RZMO & _VAR_VCD_PREJ_M0_M1_RZMO

In [191]:
df10 = df10.withColumn(f'{modality}_VAR_SALDO_M0_M1_RZMO', F.col(f'{modality}_VAR_SALDO_M0_M1_MERC')/F.col(f'{modality}_SALDO_TOTAL_TT_OMNI'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [192]:
df10 = df10.withColumn(f'{modality}_VAR_VCD_PREJ_M0_M1_RZMO', F.col(f'{modality}_VAR_VCD_PREJ_M0_M1_MERC')/F.col(f'{modality}_SALDO_TOTAL_TT_OMNI'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

RZ_MEDIA_AVCR_3MESES_PARC

In [193]:
#recreating the Omni vision columns of to be overdue credit of the next three months
df10 = df10.withColumn('CRED_A_VENCER_ATE_30_OMNI', subIgnoringNaN(['CRED_A_VENCER_ATE_30_TOTAL','CRED_A_VENCER_ATE_30_MERCADO'])).replace(0.0, None, ['CRED_A_VENCER_ATE_30_OMNI'])
df10 = df10.withColumn('CRED_A_VENCER_DE_31_A_60_OMNI', subIgnoringNaN(['CRED_A_VENCER_DE_31_A_60_TOTAL','CRED_A_VENCER_DE_31_A_60_MERCADO'])).replace(0.0, None, ['CRED_A_VENCER_DE_31_A_60_OMNI'])
df10 = df10.withColumn('CRED_A_VENCER_DE_61_A_90_OMNI', subIgnoringNaN(['CRED_A_VENCER_DE_61_A_90_TOTAL','CRED_A_VENCER_DE_61_A_90_MERCADO'])).replace(0.0, None, ['CRED_A_VENCER_DE_61_A_90_OMNI'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [194]:
df10 = df10.withColumn(f'{modality}_RZ_MEDIA_AVCR_3MESES_PARC', round(df10[f'{modality}_MEDIA_AVCR_3MESES']/avgIgnoringNaN(['CRED_A_VENCER_ATE_30_OMNI','CRED_A_VENCER_DE_31_A_60_OMNI','CRED_A_VENCER_DE_61_A_90_OMNI']).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [195]:
df10 = df10.withColumn(f'{modality}_RZ_MEDIA_AVCR_3MESES_PARC_V2', round(df10[f'{modality}_MEDIA_AVCR_3MESES_V2']/(reduce(add, [F.col(x) for x in ['CRED_A_VENCER_ATE_30_OMNI','CRED_A_VENCER_DE_31_A_60_OMNI','CRED_A_VENCER_DE_61_A_90_OMNI']])/3).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SHARE_SALDO_TOTAL & SHARE_SALDO_TOTAL_MERC

In [196]:
lista_share = ['CRT', 'VEIC', 'CP', 'CHE', 'CNG', 'MCC', 'IMB', 'DMS']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [197]:
if modality in lista_share:
    df10 = df10.withColumn(f'{modality}_SHARE_SALDO_TOTAL',round((F.col(f'{modality}_SALDO_TOTAL')/F.col(f'{modality}_SALDO_TOTAL_TT_TOTAL')).cast(DoubleType()),4))
    df10 = df10.withColumn(f'{modality}_SHARE_SALDO_TOTAL_MERC',round((F.col(f'{modality}_SALDO_TOTAL_MERC')/F.col(f'{modality}_SALDO_TOTAL_TT_TOTAL')).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SHARE_VCD & SHARE_VCD_MERC

In [198]:
if modality in lista_share:
    df10 = df10.withColumn(f'{modality}_SHARE_VCD',round((F.col(f'{modality}_VCD')/F.col(f'{modality}_SALDO_VCD_TOTAL_TT_TOTAL')).cast(DoubleType()),4))
    df10 = df10.withColumn(f'{modality}_SHARE_VCD_MERC',round((F.col(f'{modality}_VCD_MERC')/F.col(f'{modality}_SALDO_VCD_TOTAL_TT_TOTAL')).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SHARE_AVCR & SHARE_AVCR_MERC

In [199]:
if modality in lista_share:
    df10 = df10.withColumn(f'{modality}_SHARE_AVCR',round((F.col(f'{modality}_AVCR')/F.col(f'{modality}_SALDO_AVCR_TOTAL_TT_TOTAL')).cast(DoubleType()),4))
    df10 = df10.withColumn(f'{modality}_SHARE_AVCR_MERC',round((F.col(f'{modality}_AVCR_MERC')/F.col(f'{modality}_SALDO_AVCR_TOTAL_TT_TOTAL')).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SHARE_PREJ & SHARE_PREJ_MERC

In [200]:
if modality in lista_share:
    df10 = df10.withColumn(f'{modality}_SHARE_PREJ',round((F.col(f'{modality}_PREJ')/F.col(f'{modality}_SALDO_PREJUIZO_TOTAL_TT_TOTAL')).cast(DoubleType()),4))
    df10 = df10.withColumn(f'{modality}_SHARE_PREJ_MERC',round((F.col(f'{modality}_PREJ_MERC')/F.col(f'{modality}_SALDO_PREJUIZO_TOTAL_TT_TOTAL')).cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SHARE_OMNI_SALDO_TOTAL & SHARE_OMNI_VCD & SHARE_OMNI_AVCR & SHARE_OMNI_PREJUIZO

In [201]:
if modality in ['TT', 'CRT', 'VEIC', 'CP', 'CHE', 'CNG', 'MCC', 'IMB', 'DMS']:
    df10 = df10.withColumn(f'{modality}_SHARE_OMNI_SALDO_TOTAL',round((F.col(f'{modality}_SALDO_TOTAL_TT_OMNI')/F.col(f'{modality}_SALDO_TOTAL_TT_TOTAL')).cast(DoubleType()),4))
    df10 = df10.withColumn(f'{modality}_SHARE_OMNI_VCD',round((F.col(f'{modality}_SALDO_VCD_TOTAL_TT_OMNI')/F.col(f'{modality}_SALDO_VCD_TOTAL_TT_TOTAL')).cast(DoubleType()),4))
    df10 = df10.withColumn(f'{modality}_SHARE_OMNI_AVCR',round((F.col(f'{modality}_SALDO_AVCR_TOTAL_TT_OMNI')/F.col(f'{modality}_SALDO_AVCR_TOTAL_TT_TOTAL')).cast(DoubleType()),4))
    df10 = df10.withColumn(f'{modality}_SHARE_OMNI_PREJUIZO',round((F.col(f'{modality}_SALDO_PREJUIZO_TOTAL_TT_OMNI')/F.col(f'{modality}_SALDO_PREJUIZO_TOTAL_TT_TOTAL')).cast(DoubleType()),4))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [202]:
for column in df10.columns:
    if column.endswith('M0_M1') or column.endswith('M0_M1_MERC') or column.endswith('M0_M1_RZMO'):
        df10 = df10.withColumn(column, F.round(column, 4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [203]:
df10 = df10.na.fill(value=0)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [204]:
lista_em_comum = ['VEIC', 'CP', 'CNG', 'MCC', 'IMB', 'DMS']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [205]:
if modality in lista_limite_total_modalidade:
    df11 = df10.select(
    "DTBASE",
    "CD",
    "M0",
    "M1",
    "M2",
    "M3",
    "M4",
    "M5",
    f"{modality}_MAIOR_ATRASO_VCD",
    f"{modality}_MAIOR_ATRASO_VCD_MERC",
    f"{modality}_PREJ_ATE_12M",
    f"{modality}_PREJ_ATE_12M_MERC",
    f"{modality}_PREJ_ATE_12M_RZMO",
    f"{modality}_PREJ_12_48M",
    f"{modality}_PREJ_12_48M_MERC",
    f"{modality}_PREJ_12_48M_RZMO",
    f"{modality}_VCD_E_AVCR_TOTAL",
    f"{modality}_VCD_E_AVCR_MERC",
    f"{modality}_VCD_E_AVCR_RZMO",
    f"{modality}_VCD",
    f"{modality}_VCD_MERC",
    f"{modality}_VCD_RZMO",
    f"{modality}_RZ_VCD_POR_VCD_E_AVCR",
    f"{modality}_RZ_VCD_POR_VCD_E_AVCR_MERC",
    f"{modality}_RZ_VCD_PREJ_POR_DIVIDA",
    f"{modality}_RZ_VCD_PREJ_POR_DIVIDA_MERC",
    f"{modality}_MAIOR_PRAZO_AVCR",
    f"{modality}_MAIOR_PRAZO_AVCR_MERC",
    f"{modality}_RZ_AVCR_ATE90_POR_AVCR",
    f"{modality}_RZ_AVCR_ATE90_POR_AVCR_MERC",
    f"{modality}_RZ_AVCR_91_180_POR_AVCR",
    f"{modality}_RZ_AVCR_91_180_POR_AVCR_MERC",
    f"{modality}_RZ_AVCR_181_360_POR_AVCR",
    f"{modality}_RZ_AVCR_181_360_POR_AVCR_MERC",
    f"{modality}_RZ_AVCR_361_720_POR_AVCR",
    f"{modality}_RZ_AVCR_361_720_POR_AVCR_MERC",
    f"{modality}_RZ_AVCR_721_1080_POR_AVCR",
    f"{modality}_RZ_AVCR_721_1080_POR_AVCR_MERC",
    f"{modality}_RZ_AVCR_1081_1800_POR_AVCR",
    f"{modality}_RZ_AVCR_1081_1800_POR_AVCR_MERC",
    f"{modality}_RZ_AVCR_ACI1800_POR_AVCR",
    f"{modality}_RZ_AVCR_ACI1800_POR_AVCR_MERC",
    f"{modality}_AVCR_ATE90",
    f"{modality}_AVCR_ATE90_MERC",
    f"{modality}_AVCR_ATE90_RZMO",
    f"{modality}_AVCR_91_180",
    f"{modality}_AVCR_91_180_MERC",
    f"{modality}_AVCR_91_180_RZMO",
    f"{modality}_AVCR_181_360",
    f"{modality}_AVCR_181_360_MERC",
    f"{modality}_AVCR_181_360_RZMO",
    f"{modality}_AVCR_361_720",
    f"{modality}_AVCR_361_720_MERC",
    f"{modality}_AVCR_361_720_RZMO",
    f"{modality}_AVCR_721_1080",
    f"{modality}_AVCR_721_1080_MERC",
    f"{modality}_AVCR_721_1080_RZMO",
    f"{modality}_AVCR_1081_1800",
    f"{modality}_AVCR_1081_1800_MERC",
    f"{modality}_AVCR_1081_1800_RZMO",
    f"{modality}_AVCR_ACI1800",
    f"{modality}_AVCR_ACI1800_MERC",
    f"{modality}_AVCR_ACI1800_RZMO",
    f"{modality}_RZ_VCD_ATE30_POR_VCD",
    f"{modality}_RZ_VCD_ATE30_POR_VCD_MERC",
    f"{modality}_RZ_VCD_31_60_POR_VCD",
    f"{modality}_RZ_VCD_31_60_POR_VCD_MERC",
    f"{modality}_RZ_VCD_61_90_POR_VCD",
    f"{modality}_RZ_VCD_61_90_POR_VCD_MERC",
    f"{modality}_RZ_VCD_91_180_POR_VCD",
    f"{modality}_RZ_VCD_91_180_POR_VCD_MERC",
    f"{modality}_RZ_VCD_181_360_POR_VCD",
    f"{modality}_RZ_VCD_181_360_POR_VCD_MERC",
    f"{modality}_VCD_ATE30",
    f"{modality}_VCD_ATE30_MERC",
    f"{modality}_VCD_ATE30_RZMO",
    f"{modality}_VCD_31_60",
    f"{modality}_VCD_31_60_MERC",
    f"{modality}_VCD_31_60_RZMO",
    f"{modality}_VCD_61_90",
    f"{modality}_VCD_61_90_MERC",
    f"{modality}_VCD_61_90_RZMO",
    f"{modality}_VCD_91_180",
    f"{modality}_VCD_91_180_MERC",
    f"{modality}_VCD_91_180_RZMO",
    f"{modality}_VCD_181_360",
    f"{modality}_VCD_181_360_MERC",
    f"{modality}_VCD_181_360_RZMO",
    f"{modality}_VAR_MAIOR_ATRASO_M0_M1",
    f"{modality}_VAR_MAIOR_ATRASO_M0_M1_MERC",
    f"{modality}_VAR_SALDO_M0_M1",
    f"{modality}_VAR_SALDO_M0_M1_MERC",
    f"{modality}_VAR_SALDO_M0_M1_RZMO",
    f"{modality}_VAR_VCD_PREJ_M0_M1",
    f"{modality}_VAR_VCD_PREJ_M0_M1_MERC",
    f"{modality}_VAR_VCD_PREJ_M0_M1_RZMO",
    f"{modality}_MEDIA_AVCR_3MESES",
    f"{modality}_MEDIA_AVCR_3MESES_MERC",
    f"{modality}_RZ_MEDIA_AVCR_3MESES_PARC",
    f"{modality}_MEDIA_AVCR_3MESES_V2",
    f"{modality}_MEDIA_AVCR_3MESES_MERC_V2",
    f"{modality}_RZ_MEDIA_AVCR_3MESES_PARC_V2",
    f"{modality}_LMT_DISPONIVEL",
    f"{modality}_LMT_DISPONIVEL_MERC",
    f"{modality}_LMT_DISPONIVEL_RZMO",
    f"{modality}_LMT_TOTAL",
    f"{modality}_LMT_TOTAL_MERC",
    f"{modality}_LMT_TOTAL_RZMO",
    f"{modality}_RZ_LMT_TOTAL_QTD_INST_FIN",
    f"{modality}_RZ_LMT_TOTAL_QTD_OPERAC",
    f"{modality}_PERC_LIMITE_DISPONIVEL",
    f"{modality}_PERC_LIMITE_DISPONIVEL_MERC",
    f"{modality}_PERC_LIMITE_DISP_3M",
    f"{modality}_PERC_LIMITE_DISP_3M_MERC",
    f"{modality}_PERC_LIMITE_DISP_3M_V2",
    f"{modality}_PERC_LIMITE_DISP_3M_MERC_V2",
    f"{modality}_SALDO_TOTAL",
    f"{modality}_SALDO_TOTAL_MERC",
    f"{modality}_SALDO_TOTAL_RZMO",
    f"{modality}_PREJ",
    f"{modality}_PREJ_MERC",
    f"{modality}_PREJ_RZMO",
    f"{modality}_AVCR",
    f"{modality}_AVCR_MERC",
    f"{modality}_AVCR_RZMO",
    f"{modality}_SHARE_SALDO_TOTAL",
    f"{modality}_SHARE_SALDO_TOTAL_MERC",
    f"{modality}_SHARE_VCD",
    f"{modality}_SHARE_VCD_MERC",
    f"{modality}_SHARE_AVCR",
    f"{modality}_SHARE_AVCR_MERC",
    f"{modality}_SHARE_PREJ",
    f"{modality}_SHARE_PREJ_MERC",
    f"{modality}_SHARE_OMNI_SALDO_TOTAL",
    f"{modality}_SHARE_OMNI_VCD",
    f"{modality}_SHARE_OMNI_AVCR",
    f"{modality}_SHARE_OMNI_PREJUIZO",
    f"{modality}_RZ_SALDO_M1_M0",
    f"{modality}_RZ_SALDO_M1_M0_MERC",
    f"{modality}_RZ_VCD_M1_M0",
    f"{modality}_RZ_VCD_M1_M0_MERC",
    f"{modality}_RZ_AVCR_M1_M0",
    f"{modality}_RZ_AVCR_M1_M0_MERC",
    f"{modality}_RZ_PREJ_M1_M0",
    f"{modality}_RZ_PREJ_M1_M0_MERC")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [206]:
if modality in 'TT':
	df11=df10.select(
	"DTBASE",
	"CD",
    "M0",
    "M1",
    "M2",
    "M3",
    "M4",
    "M5",
    f"{modality}_MAIOR_ATRASO_VCD",
    f"{modality}_MAIOR_ATRASO_VCD_MERC",
    f"{modality}_PREJ_ATE_12M",
    f"{modality}_PREJ_ATE_12M_MERC",
    f"{modality}_PREJ_ATE_12M_RZMO",
    f"{modality}_PREJ_12_48M",
    f"{modality}_PREJ_12_48M_MERC",
    f"{modality}_PREJ_12_48M_RZMO",
    f"{modality}_VCD_E_AVCR_TOTAL",
    f"{modality}_VCD_E_AVCR_MERC",
    f"{modality}_VCD_E_AVCR_RZMO",
    f"{modality}_VCD",
    f"{modality}_VCD_MERC",
    f"{modality}_VCD_RZMO",
    f"{modality}_RZ_VCD_POR_VCD_E_AVCR",
    f"{modality}_RZ_VCD_POR_VCD_E_AVCR_MERC",
    f"{modality}_RZ_VCD_PREJ_POR_DIVIDA",
    f"{modality}_RZ_VCD_PREJ_POR_DIVIDA_MERC",
    f"{modality}_MAIOR_PRAZO_AVCR",
    f"{modality}_MAIOR_PRAZO_AVCR_MERC",
    f"{modality}_RZ_AVCR_ATE90_POR_AVCR",
    f"{modality}_RZ_AVCR_ATE90_POR_AVCR_MERC",
    f"{modality}_RZ_AVCR_91_180_POR_AVCR",
    f"{modality}_RZ_AVCR_91_180_POR_AVCR_MERC",
    f"{modality}_RZ_AVCR_181_360_POR_AVCR",
    f"{modality}_RZ_AVCR_181_360_POR_AVCR_MERC",
    f"{modality}_RZ_AVCR_361_720_POR_AVCR",
    f"{modality}_RZ_AVCR_361_720_POR_AVCR_MERC",
    f"{modality}_RZ_AVCR_721_1080_POR_AVCR",
    f"{modality}_RZ_AVCR_721_1080_POR_AVCR_MERC",
    f"{modality}_RZ_AVCR_1081_1800_POR_AVCR",
    f"{modality}_RZ_AVCR_1081_1800_POR_AVCR_MERC",
    f"{modality}_RZ_AVCR_ACI1800_POR_AVCR",
    f"{modality}_RZ_AVCR_ACI1800_POR_AVCR_MERC",
    f"{modality}_AVCR_ATE90",
    f"{modality}_AVCR_ATE90_MERC",
    f"{modality}_AVCR_ATE90_RZMO",
    f"{modality}_AVCR_91_180",
    f"{modality}_AVCR_91_180_MERC",
    f"{modality}_AVCR_91_180_RZMO",
    f"{modality}_AVCR_181_360",
    f"{modality}_AVCR_181_360_MERC",
    f"{modality}_AVCR_181_360_RZMO",
    f"{modality}_AVCR_361_720",
    f"{modality}_AVCR_361_720_MERC",
    f"{modality}_AVCR_361_720_RZMO",
    f"{modality}_AVCR_721_1080",
    f"{modality}_AVCR_721_1080_MERC",
    f"{modality}_AVCR_721_1080_RZMO",
    f"{modality}_AVCR_1081_1800",
    f"{modality}_AVCR_1081_1800_MERC",
    f"{modality}_AVCR_1081_1800_RZMO",
    f"{modality}_AVCR_ACI1800",
    f"{modality}_AVCR_ACI1800_MERC",
    f"{modality}_AVCR_ACI1800_RZMO",
    f"{modality}_RZ_VCD_ATE30_POR_VCD",
    f"{modality}_RZ_VCD_ATE30_POR_VCD_MERC",
    f"{modality}_RZ_VCD_31_60_POR_VCD",
    f"{modality}_RZ_VCD_31_60_POR_VCD_MERC",
    f"{modality}_RZ_VCD_61_90_POR_VCD",
    f"{modality}_RZ_VCD_61_90_POR_VCD_MERC",
    f"{modality}_RZ_VCD_91_180_POR_VCD",
    f"{modality}_RZ_VCD_91_180_POR_VCD_MERC",
    f"{modality}_RZ_VCD_181_360_POR_VCD",
    f"{modality}_RZ_VCD_181_360_POR_VCD_MERC",
    f"{modality}_VCD_ATE30",
    f"{modality}_VCD_ATE30_MERC",
    f"{modality}_VCD_ATE30_RZMO",
    f"{modality}_VCD_31_60",
    f"{modality}_VCD_31_60_MERC",
    f"{modality}_VCD_31_60_RZMO",
    f"{modality}_VCD_61_90",
    f"{modality}_VCD_61_90_MERC",
    f"{modality}_VCD_61_90_RZMO",
    f"{modality}_VCD_91_180",
    f"{modality}_VCD_91_180_MERC",
    f"{modality}_VCD_91_180_RZMO",
    f"{modality}_VCD_181_360",
    f"{modality}_VCD_181_360_MERC",
    f"{modality}_VCD_181_360_RZMO",
    f"{modality}_VAR_MAIOR_ATRASO_M0_M1",
    f"{modality}_VAR_MAIOR_ATRASO_M0_M1_MERC",
    f"{modality}_VAR_SALDO_M0_M1",
    f"{modality}_VAR_SALDO_M0_M1_MERC",
    f"{modality}_VAR_SALDO_M0_M1_RZMO",
    f"{modality}_VAR_VCD_PREJ_M0_M1",
    f"{modality}_VAR_VCD_PREJ_M0_M1_MERC",
    f"{modality}_VAR_VCD_PREJ_M0_M1_RZMO",
    f"{modality}_MEDIA_AVCR_3MESES",
    f"{modality}_MEDIA_AVCR_3MESES_MERC",
    f"{modality}_RZ_MEDIA_AVCR_3MESES_PARC",
    f"{modality}_MEDIA_AVCR_3MESES_V2",
    f"{modality}_MEDIA_AVCR_3MESES_MERC_V2",
    f"{modality}_RZ_MEDIA_AVCR_3MESES_PARC_V2",
    "QTDIF",
    "QTDOP",
    f"{modality}_TEMPO_RELACIONAMENTO",
    f"{modality}_SALDO_TOTAL",
    f"{modality}_SALDO_TOTAL_MERC",
    f"{modality}_SALDO_TOTAL_RZMO",
    f"{modality}_PREJ",
    f"{modality}_PREJ_MERC",
    f"{modality}_PREJ_RZMO",
    f"{modality}_AVCR",
    f"{modality}_AVCR_MERC",
    f"{modality}_AVCR_RZMO",
    f"{modality}_SHARE_OMNI_SALDO_TOTAL",
    f"{modality}_SHARE_OMNI_VCD",
    f"{modality}_SHARE_OMNI_AVCR",
    f"{modality}_SHARE_OMNI_PREJUIZO",
    f"{modality}_RZ_SALDO_M1_M0",
    f"{modality}_RZ_SALDO_M1_M0_MERC",
    f"{modality}_RZ_VCD_M1_M0",
    f"{modality}_RZ_VCD_M1_M0_MERC",
    f"{modality}_RZ_AVCR_M1_M0",
    f"{modality}_RZ_AVCR_M1_M0_MERC",
    f"{modality}_RZ_PREJ_M1_M0",
    f"{modality}_RZ_PREJ_M1_M0_MERC"
	)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [207]:
if modality in lista_em_comum:
	df11=df10.select(
	"DTBASE",
	"CD",
    "M0",
    "M1",
    "M2",
    "M3",
    "M4",
    "M5",
	f"{modality}_MAIOR_ATRASO_VCD",
	f"{modality}_MAIOR_ATRASO_VCD_MERC",
	f"{modality}_PREJ_ATE_12M",
	f"{modality}_PREJ_ATE_12M_MERC",
	f"{modality}_PREJ_ATE_12M_RZMO",
	f"{modality}_PREJ_12_48M",
	f"{modality}_PREJ_12_48M_MERC",
	f"{modality}_PREJ_12_48M_RZMO",
	f"{modality}_VCD_E_AVCR_TOTAL",
	f"{modality}_VCD_E_AVCR_MERC",
	f"{modality}_VCD_E_AVCR_RZMO",
	f"{modality}_VCD",
	f"{modality}_VCD_MERC",
	f"{modality}_VCD_RZMO",
	f"{modality}_RZ_VCD_POR_VCD_E_AVCR",
	f"{modality}_RZ_VCD_POR_VCD_E_AVCR_MERC",
	f"{modality}_RZ_VCD_PREJ_POR_DIVIDA",
	f"{modality}_RZ_VCD_PREJ_POR_DIVIDA_MERC",
	f"{modality}_MAIOR_PRAZO_AVCR",
	f"{modality}_MAIOR_PRAZO_AVCR_MERC",
	f"{modality}_RZ_AVCR_ATE90_POR_AVCR",
	f"{modality}_RZ_AVCR_ATE90_POR_AVCR_MERC",
	f"{modality}_RZ_AVCR_91_180_POR_AVCR",
	f"{modality}_RZ_AVCR_91_180_POR_AVCR_MERC",
	f"{modality}_RZ_AVCR_181_360_POR_AVCR",
	f"{modality}_RZ_AVCR_181_360_POR_AVCR_MERC",
	f"{modality}_RZ_AVCR_361_720_POR_AVCR",
	f"{modality}_RZ_AVCR_361_720_POR_AVCR_MERC",
	f"{modality}_RZ_AVCR_721_1080_POR_AVCR",
	f"{modality}_RZ_AVCR_721_1080_POR_AVCR_MERC",
	f"{modality}_RZ_AVCR_1081_1800_POR_AVCR",
	f"{modality}_RZ_AVCR_1081_1800_POR_AVCR_MERC",
	f"{modality}_RZ_AVCR_ACI1800_POR_AVCR",
	f"{modality}_RZ_AVCR_ACI1800_POR_AVCR_MERC",
	f"{modality}_AVCR_ATE90",
	f"{modality}_AVCR_ATE90_MERC",
	f"{modality}_AVCR_ATE90_RZMO",
	f"{modality}_AVCR_91_180",
	f"{modality}_AVCR_91_180_MERC",
	f"{modality}_AVCR_91_180_RZMO",
	f"{modality}_AVCR_181_360",
	f"{modality}_AVCR_181_360_MERC",
	f"{modality}_AVCR_181_360_RZMO",
	f"{modality}_AVCR_361_720",
	f"{modality}_AVCR_361_720_MERC",
	f"{modality}_AVCR_361_720_RZMO",
	f"{modality}_AVCR_721_1080",
	f"{modality}_AVCR_721_1080_MERC",
	f"{modality}_AVCR_721_1080_RZMO",
	f"{modality}_AVCR_1081_1800",
	f"{modality}_AVCR_1081_1800_MERC",
	f"{modality}_AVCR_1081_1800_RZMO",
	f"{modality}_AVCR_ACI1800",
	f"{modality}_AVCR_ACI1800_MERC",
	f"{modality}_AVCR_ACI1800_RZMO",
	f"{modality}_RZ_VCD_ATE30_POR_VCD",
	f"{modality}_RZ_VCD_ATE30_POR_VCD_MERC",
	f"{modality}_RZ_VCD_31_60_POR_VCD",
	f"{modality}_RZ_VCD_31_60_POR_VCD_MERC",
	f"{modality}_RZ_VCD_61_90_POR_VCD",
	f"{modality}_RZ_VCD_61_90_POR_VCD_MERC",
	f"{modality}_RZ_VCD_91_180_POR_VCD",
	f"{modality}_RZ_VCD_91_180_POR_VCD_MERC",
	f"{modality}_RZ_VCD_181_360_POR_VCD",
	f"{modality}_RZ_VCD_181_360_POR_VCD_MERC",
	f"{modality}_VCD_ATE30",
	f"{modality}_VCD_ATE30_MERC",
	f"{modality}_VCD_ATE30_RZMO",
	f"{modality}_VCD_31_60",
	f"{modality}_VCD_31_60_MERC",
	f"{modality}_VCD_31_60_RZMO",
	f"{modality}_VCD_61_90",
	f"{modality}_VCD_61_90_MERC",
	f"{modality}_VCD_61_90_RZMO",
	f"{modality}_VCD_91_180",
	f"{modality}_VCD_91_180_MERC",
	f"{modality}_VCD_91_180_RZMO",
	f"{modality}_VCD_181_360",
	f"{modality}_VCD_181_360_MERC",
	f"{modality}_VCD_181_360_RZMO",
	f"{modality}_VAR_MAIOR_ATRASO_M0_M1",
	f"{modality}_VAR_MAIOR_ATRASO_M0_M1_MERC",
	f"{modality}_VAR_SALDO_M0_M1",
	f"{modality}_VAR_SALDO_M0_M1_MERC",
	f"{modality}_VAR_SALDO_M0_M1_RZMO",
	f"{modality}_VAR_VCD_PREJ_M0_M1",
	f"{modality}_VAR_VCD_PREJ_M0_M1_MERC",
	f"{modality}_VAR_VCD_PREJ_M0_M1_RZMO",
	f"{modality}_MEDIA_AVCR_3MESES",
	f"{modality}_MEDIA_AVCR_3MESES_MERC",
	f"{modality}_RZ_MEDIA_AVCR_3MESES_PARC",
	f"{modality}_MEDIA_AVCR_3MESES_V2",
	f"{modality}_MEDIA_AVCR_3MESES_MERC_V2",
	f"{modality}_RZ_MEDIA_AVCR_3MESES_PARC_V2",
	f"{modality}_SALDO_TOTAL",
	f"{modality}_SALDO_TOTAL_MERC",
	f"{modality}_SALDO_TOTAL_RZMO",
	f"{modality}_PREJ",
	f"{modality}_PREJ_MERC",
	f"{modality}_PREJ_RZMO",
	f"{modality}_AVCR",
	f"{modality}_AVCR_MERC",
	f"{modality}_AVCR_RZMO",
	f"{modality}_SHARE_SALDO_TOTAL",
	f"{modality}_SHARE_SALDO_TOTAL_MERC",
	f"{modality}_SHARE_VCD",
	f"{modality}_SHARE_VCD_MERC",
	f"{modality}_SHARE_AVCR",
	f"{modality}_SHARE_AVCR_MERC",
	f"{modality}_SHARE_PREJ",
	f"{modality}_SHARE_PREJ_MERC",
	f"{modality}_SHARE_OMNI_SALDO_TOTAL",
	f"{modality}_SHARE_OMNI_VCD",
	f"{modality}_SHARE_OMNI_AVCR",
	f"{modality}_SHARE_OMNI_PREJUIZO",
	f"{modality}_RZ_SALDO_M1_M0",
	f"{modality}_RZ_SALDO_M1_M0_MERC",
	f"{modality}_RZ_VCD_M1_M0",
	f"{modality}_RZ_VCD_M1_M0_MERC",
	f"{modality}_RZ_AVCR_M1_M0",
	f"{modality}_RZ_AVCR_M1_M0_MERC",
	f"{modality}_RZ_PREJ_M1_M0",
    f"{modality}_RZ_PREJ_M1_M0_MERC"
	)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [208]:
if modality in 'OLM':
	df11=df10.select(
	"DTBASE",
	"CD",
    "M0",
    "M1",
    "M2",
    "M3",
    "M4",
    "M5",
	f"{modality}_LMT_DISPONIVEL",
	f"{modality}_LMT_DISPONIVEL_MERC",
	f"{modality}_LMT_DISPONIVEL_RZMO"
	)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [209]:
del(df10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [210]:
df11.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- DTBASE: integer (nullable = true)
 |-- CD: string (nullable = true)
 |-- M0: date (nullable = true)
 |-- M1: date (nullable = true)
 |-- M2: date (nullable = true)
 |-- M3: date (nullable = true)
 |-- M4: date (nullable = true)
 |-- M5: date (nullable = true)
 |-- TT_MAIOR_ATRASO_VCD: integer (nullable = true)
 |-- TT_MAIOR_ATRASO_VCD_MERC: integer (nullable = true)
 |-- TT_PREJ_ATE_12M: double (nullable = false)
 |-- TT_PREJ_ATE_12M_MERC: double (nullable = false)
 |-- TT_PREJ_ATE_12M_RZMO: double (nullable = false)
 |-- TT_PREJ_12_48M: double (nullable = false)
 |-- TT_PREJ_12_48M_MERC: double (nullable = false)
 |-- TT_PREJ_12_48M_RZMO: double (nullable = false)
 |-- TT_VCD_E_AVCR_TOTAL: double (nullable = false)
 |-- TT_VCD_E_AVCR_MERC: double (nullable = false)
 |-- TT_VCD_E_AVCR_RZMO: double (nullable = false)
 |-- TT_VCD: double (nullable = false)
 |-- TT_VCD_MERC: double (nullable = false)
 |-- TT_VCD_RZMO: double (nullable = false)
 |-- TT_RZ_VCD_POR_VCD_E_AVCR: doub

In [211]:
df11.write.format("parquet").mode("Overwrite").save(f"s3://modelagem/scr/m0-processed-data/{modality}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [212]:
df_visoes_mensais = spark.read.parquet(f"s3://modelagem/scr/m0-processed-data/{modality}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [213]:
if modality in lista_limite_total_modalidade:
	df_COLUNAS_MX = df_visoes_mensais.select(
	"CD",
	"M0",
	f"{modality}_MAIOR_ATRASO_VCD",
	f"{modality}_MAIOR_ATRASO_VCD_MERC",
	f"{modality}_VCD_E_AVCR_TOTAL",
	f"{modality}_VCD_E_AVCR_MERC",
	f"{modality}_VCD_E_AVCR_RZMO",
	f"{modality}_VCD",
	f"{modality}_VCD_MERC",
	f"{modality}_VCD_RZMO",
	f"{modality}_RZ_VCD_POR_VCD_E_AVCR",
	f"{modality}_RZ_VCD_POR_VCD_E_AVCR_MERC",
	f"{modality}_RZ_VCD_PREJ_POR_DIVIDA",
	f"{modality}_RZ_VCD_PREJ_POR_DIVIDA_MERC",
	f"{modality}_MAIOR_PRAZO_AVCR",
	f"{modality}_MAIOR_PRAZO_AVCR_MERC",
	f"{modality}_LMT_DISPONIVEL",
	f"{modality}_LMT_DISPONIVEL_MERC",
	f"{modality}_LMT_DISPONIVEL_RZMO",
	f"{modality}_LMT_TOTAL",
	f"{modality}_LMT_TOTAL_MERC",
	f"{modality}_LMT_TOTAL_RZMO",
	f"{modality}_RZ_LMT_TOTAL_QTD_INST_FIN",
	f"{modality}_RZ_LMT_TOTAL_QTD_OPERAC",
	f"{modality}_PERC_LIMITE_DISPONIVEL",
	f"{modality}_PERC_LIMITE_DISPONIVEL_MERC",
	f"{modality}_SALDO_TOTAL",
	f"{modality}_SALDO_TOTAL_MERC",
	f"{modality}_SALDO_TOTAL_RZMO",
	f"{modality}_PREJ",
	f"{modality}_PREJ_MERC",
	f"{modality}_PREJ_RZMO",
	f"{modality}_AVCR_MERC",
	f"{modality}_AVCR_RZMO",
	f"{modality}_SHARE_SALDO_TOTAL",
	f"{modality}_SHARE_SALDO_TOTAL_MERC",
	f"{modality}_SHARE_VCD",
	f"{modality}_SHARE_VCD_MERC",
	f"{modality}_SHARE_AVCR",
	f"{modality}_SHARE_AVCR_MERC",
	f"{modality}_SHARE_PREJ",
	f"{modality}_SHARE_PREJ_MERC",
	f"{modality}_SHARE_OMNI_SALDO_TOTAL",
	f"{modality}_SHARE_OMNI_VCD",
	f"{modality}_SHARE_OMNI_AVCR",
	f"{modality}_SHARE_OMNI_PREJUIZO")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [214]:
if modality in lista_em_comum:
	df_COLUNAS_MX = df_visoes_mensais.select(
	"CD",
	"M0",
	f"{modality}_MAIOR_ATRASO_VCD",
	f"{modality}_MAIOR_ATRASO_VCD_MERC",
	f"{modality}_VCD_E_AVCR_TOTAL",
	f"{modality}_VCD_E_AVCR_MERC",
	f"{modality}_VCD_E_AVCR_RZMO",
	f"{modality}_VCD",
	f"{modality}_VCD_MERC",
	f"{modality}_VCD_RZMO",
	f"{modality}_RZ_VCD_POR_VCD_E_AVCR",
	f"{modality}_RZ_VCD_POR_VCD_E_AVCR_MERC",
	f"{modality}_RZ_VCD_PREJ_POR_DIVIDA",
	f"{modality}_RZ_VCD_PREJ_POR_DIVIDA_MERC",
	f"{modality}_MAIOR_PRAZO_AVCR",
	f"{modality}_MAIOR_PRAZO_AVCR_MERC",
	f"{modality}_SALDO_TOTAL",
	f"{modality}_SALDO_TOTAL_MERC",
	f"{modality}_SALDO_TOTAL_RZMO",
	f"{modality}_PREJ",
	f"{modality}_PREJ_MERC",
	f"{modality}_PREJ_RZMO",
	f"{modality}_AVCR_MERC",
	f"{modality}_AVCR_RZMO",
	f"{modality}_SHARE_SALDO_TOTAL",
	f"{modality}_SHARE_SALDO_TOTAL_MERC",
	f"{modality}_SHARE_VCD",
	f"{modality}_SHARE_VCD_MERC",
	f"{modality}_SHARE_AVCR",
	f"{modality}_SHARE_AVCR_MERC",
	f"{modality}_SHARE_PREJ",
	f"{modality}_SHARE_PREJ_MERC",
	f"{modality}_SHARE_OMNI_SALDO_TOTAL",
	f"{modality}_SHARE_OMNI_VCD",
	f"{modality}_SHARE_OMNI_AVCR",
	f"{modality}_SHARE_OMNI_PREJUIZO")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [215]:
if modality in 'TT':
	df_COLUNAS_MX = df_visoes_mensais.select(
	"CD",
	"M0",
	f"{modality}_MAIOR_ATRASO_VCD",
	f"{modality}_MAIOR_ATRASO_VCD_MERC",
	f"{modality}_VCD_E_AVCR_TOTAL",
	f"{modality}_VCD_E_AVCR_MERC",
	f"{modality}_VCD_E_AVCR_RZMO",
	f"{modality}_VCD",
	f"{modality}_VCD_MERC",
	f"{modality}_VCD_RZMO",
	f"{modality}_RZ_VCD_POR_VCD_E_AVCR",
	f"{modality}_RZ_VCD_POR_VCD_E_AVCR_MERC",
	f"{modality}_RZ_VCD_PREJ_POR_DIVIDA",
	f"{modality}_RZ_VCD_PREJ_POR_DIVIDA_MERC",
	f"{modality}_MAIOR_PRAZO_AVCR",
	f"{modality}_MAIOR_PRAZO_AVCR_MERC",
	f"{modality}_SALDO_TOTAL",
	f"{modality}_SALDO_TOTAL_MERC",
	f"{modality}_SALDO_TOTAL_RZMO",
	f"{modality}_PREJ",
	f"{modality}_PREJ_MERC",
	f"{modality}_PREJ_RZMO",
	f"{modality}_AVCR_MERC",
	f"{modality}_AVCR_RZMO",
	f"{modality}_SHARE_OMNI_SALDO_TOTAL",
	f"{modality}_SHARE_OMNI_VCD",
	f"{modality}_SHARE_OMNI_AVCR",
	f"{modality}_SHARE_OMNI_PREJUIZO")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [216]:
if modality in 'OLM':
	df_COLUNAS_MX = df_visoes_mensais.select(
	"CD",
	"M0",
	f"{modality}_LMT_DISPONIVEL",
	f"{modality}_LMT_DISPONIVEL_MERC",
	f"{modality}_LMT_DISPONIVEL_RZMO")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [217]:
df_m1 = df_COLUNAS_MX
df_m2 = df_COLUNAS_MX
df_m3 = df_COLUNAS_MX
df_m4 = df_COLUNAS_MX
df_m5 = df_COLUNAS_MX

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [218]:
for column in df_m1.columns[2:]:
    df_m1 = df_m1.withColumnRenamed(column, column+'_M1')
df_m1 = df_m1.withColumnRenamed('M0', 'M1')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [219]:
for column in df_m2.columns[2:]:
    df_m2 = df_m2.withColumnRenamed(column, column+'_M2')
df_m2 = df_m2.withColumnRenamed('M0', 'M2')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [220]:
for column in df_m3.columns[2:]:
    df_m3 = df_m3.withColumnRenamed(column, column+'_M3')
df_m3 = df_m3.withColumnRenamed('M0', 'M3')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [221]:
for column in df_m4.columns[2:]:
    df_m4 = df_m4.withColumnRenamed(column, column+'_M4')
df_m4 = df_m4.withColumnRenamed('M0', 'M4')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [222]:
for column in df_m5.columns[2:]:
    df_m5 = df_m5.withColumnRenamed(column, column+'_M5')
df_m5 = df_m5.withColumnRenamed('M0', 'M5')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [223]:
df_visoes_mensais = df_visoes_mensais.join(df_m1,["M1","CD"],"left")# join entre M0 e M1
df_visoes_mensais = df_visoes_mensais.join(df_m2,["M2","CD"],"left")# join entre M0 e M2
df_visoes_mensais = df_visoes_mensais.join(df_m3,["M3","CD"],"left")# join entre M0 e M3
df_visoes_mensais = df_visoes_mensais.join(df_m4,["M4","CD"],"left")# join entre M0 e M4
df_visoes_mensais = df_visoes_mensais.join(df_m5,["M5","CD"],"left")# join entre M0 e M5

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [224]:
#df50 = df10

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [225]:
#df_COLUNAS_MX = df50.select("CD","M0",
#	f"{modality}_MAIOR_ATRASO_VCD",
#	f"{modality}_MAIOR_ATRASO_VCD_MERC",
#	f"{modality}_VCD_E_AVCR_TOTAL",
#	f"{modality}_VCD_E_AVCR_MERC",
#	f"{modality}_VCD_E_AVCR_RZMO",
#	f"{modality}_VCD",
#	f"{modality}_VCD_MERC",
#	f"{modality}_VCD_RZMO",
#	f"{modality}_RZ_VCD_POR_VCD_E_AVCR",
#	f"{modality}_RZ_VCD_POR_VCD_E_AVCR_MERC",
#	f"{modality}_RZ_VCD_PREJ_POR_DIVIDA",
#	f"{modality}_RZ_VCD_PREJ_POR_DIVIDA_MERC",
#	f"{modality}_MAIOR_PRAZO_AVCR",
#	f"{modality}_MAIOR_PRAZO_AVCR_MERC",
#	f"{modality}_LMT_DISPONIVEL",
#	f"{modality}_LMT_DISPONIVEL_MERC",
#	f"{modality}_LMT_DISPONIVEL_RZMO",
#	f"{modality}_LMT_TOTAL",
#	f"{modality}_LMT_TOTAL_MERC",
#	f"{modality}_LMT_TOTAL_RZMO",
#	f"{modality}_RZ_LMT_TOTAL_QTD_INST_FIN",
#	f"{modality}_RZ_LMT_TOTAL_QTD_OPERAC",
#	f"{modality}_PERC_LIMITE_DISPONIVEL",
#	f"{modality}_PERC_LIMITE_DISPONIVEL_MERC",
#	f"{modality}_SALDO_TOTAL",
#	f"{modality}_SALDO_TOTAL_MERC",
#	f"{modality}_SALDO_TOTAL_RZMO",
#	f"{modality}_PREJ",
#	f"{modality}_PREJ_MERC",
#	f"{modality}_PREJ_RZMO",
#	f"{modality}_AVCR_MERC",
#	f"{modality}_AVCR_RZMO",
#	f"{modality}_SHARE_SALDO_TOTAL",
#	f"{modality}_SHARE_SALDO_TOTAL_MERC",
#	f"{modality}_SHARE_VCD",
#	f"{modality}_SHARE_VCD_MERC",
#	f"{modality}_SHARE_AVCR",
#	f"{modality}_SHARE_AVCR_MERC",
#	f"{modality}_SHARE_PREJ",
#	f"{modality}_SHARE_PREJ_MERC",
#	f"{modality}_SHARE_OMNI_SALDO_TOTAL",
#	f"{modality}_SHARE_OMNI_VCD",
#	f"{modality}_SHARE_OMNI_AVCR",
#	f"{modality}_SHARE_OMNI_PREJUIZO")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [226]:
#for column in df50.columns[2:132]:
#    df50 = df50.withColumnRenamed(column+'_M5',column+'_DELETED')\
#    .withColumnRenamed(column+'_M4',column+'_M5')\
#    .withColumnRenamed(column+'_M3',column+'_M4')\
#    .withColumnRenamed(column+'_M2',column+'_M3')\
#    .withColumnRenamed(column+'_M1',column+'_M2')\
#    .drop(column+'_DELETED')          

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [227]:
#df_m1 = df_COLUNAS_MX

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [228]:
#for column in df_m1.columns[2:]:
#    df_m1 = df_m1.withColumnRenamed(column, column+'_M1')
#df_m1 = df_m1.withColumnRenamed('M0', 'M1')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [229]:
#df50 = df50.join(df_m1,["M1","CD"],"inner")# join entre M0 e M1

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [230]:
df_visoes_mensais.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- M5: date (nullable = true)
 |-- CD: string (nullable = true)
 |-- M4: date (nullable = true)
 |-- M3: date (nullable = true)
 |-- M2: date (nullable = true)
 |-- M1: date (nullable = true)
 |-- DTBASE: integer (nullable = true)
 |-- M0: date (nullable = true)
 |-- TT_MAIOR_ATRASO_VCD: integer (nullable = true)
 |-- TT_MAIOR_ATRASO_VCD_MERC: integer (nullable = true)
 |-- TT_PREJ_ATE_12M: double (nullable = true)
 |-- TT_PREJ_ATE_12M_MERC: double (nullable = true)
 |-- TT_PREJ_ATE_12M_RZMO: double (nullable = true)
 |-- TT_PREJ_12_48M: double (nullable = true)
 |-- TT_PREJ_12_48M_MERC: double (nullable = true)
 |-- TT_PREJ_12_48M_RZMO: double (nullable = true)
 |-- TT_VCD_E_AVCR_TOTAL: double (nullable = true)
 |-- TT_VCD_E_AVCR_MERC: double (nullable = true)
 |-- TT_VCD_E_AVCR_RZMO: double (nullable = true)
 |-- TT_VCD: double (nullable = true)
 |-- TT_VCD_MERC: double (nullable = true)
 |-- TT_VCD_RZMO: double (nullable = true)
 |-- TT_RZ_VCD_POR_VCD_E_AVCR: double (nullable

In [231]:
#df10.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [232]:
#df10.write.format("parquet").mode("append").saveAsTable(
#        path="s3://modelagem/scr/etl-variaveis/processed-data/",
#        name="modelagem.scr_cliente"
#    )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [233]:
#df_visoes_mensais.write.format("parquet").mode("append").save(f"s3://modelagem/scr/processed-data/{modality}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [234]:
df_visoes_mensais = df_visoes_mensais.drop("M0","M1","M2","M3","M4","M5")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [235]:
df_visoes_mensais.write.format("parquet").mode("Overwrite").save("s3://modelagem/scr/processed-data/HML/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [236]:
df_visoes_mensais.show(df_visoes_mensais.count(),False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+------+-------------------+------------------------+---------------+--------------------+--------------------+--------------+-------------------+-------------------+-------------------+------------------+------------------+-------+-----------+-----------+------------------------+-----------------------------+-------------------------+------------------------------+-------------------+------------------------+-------------------------+------------------------------+--------------------------+-------------------------------+---------------------------+--------------------------------+---------------------------+--------------------------------+----------------------------+---------------------------------+-----------------------------+----------------------------------+---------------------------+--------------------------------+-------------+------------------+------------------+--------------+-------------------+-------------------+---------------+--------------------+----

In [237]:
#df_visoes_mensais.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [238]:
df_visoes_mensais.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

6